<h1 align="center">Introduction to Machine Learning - 25737-2</h1>
<h4 align="center">Dr. R. Amiri</h4>
<h4 align="center">Sharif University of Technology, Spring 2024</h4>


<h4 align="center">Prject Phase-1</h4>


<h4 align="center">Seyed Hamid Ghasemi (400109768) </h4>
<h4 align="center">Parsa Novrouzinezhad (400102182) </h4>



# **libraries & Google drive**:

In [16]:
import os
import shutil
from PIL import Image
import zipfile
import time
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset, TensorDataset, ConcatDataset
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
import numpy as np
from google.colab import files
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score

In [17]:
!pip install -U -q PyDrive
!pip install -U -q pydrive2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.3 MB/s eta 0:00:00


In [18]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [19]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Implement the ResNet18 model with additional fully connected layers for CIFAR-10
class ModifiedResNet18(nn.Module):
    def __init__(self):
        super(ModifiedResNet18, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        self.resnet.fc = nn.Sequential(
            nn.Linear(self.resnet.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        return self.resnet(x)

In [20]:
# Authenticate and initialize PyDrive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# **SISA CLASS**

In [21]:
# Define SISA algorithm components
class SISA:
    def __init__(self, S, R, epochs):
        self.S = S
        self.R = R
        self.epochs = epochs
        self.models = []  # Final models for each shard
        self.slice_models = {}  # Dictionary to save models after each slice

        self.unlearning_percent = 0


        # Variables to track training and unlearning times
        self.training_time = 0
        self.unlearning_time = 0

        # Create a directory to save models
        self.model_dir = f'models_S{self.S}_R{self.R}'
        if os.path.exists(self.model_dir):
            shutil.rmtree(self.model_dir)
        os.makedirs(self.model_dir)

    def shard_data(self, data):
        # Shard the data into S subsets
        shard_size = len(data) // self.S
        shards = [Subset(data, list(range(i * shard_size, (i + 1) * shard_size))) for i in range(self.S)]
        return shards

    def slice_data(self, shard):
        # Slice each shard into R slices
        slice_size = len(shard) // self.R
        slices = [Subset(shard, list(range(i * slice_size, (i + 1) * slice_size))) for i in range(self.R)]
        return slices

    def train(self, data):
        start_time = time.time()  # Start timing
        k = 0
        shards = self.shard_data(data)
        idx_shard = 0
        for shard in shards:
            idx_shard += 1
            slices = self.slice_data(shard)
            model = ModifiedResNet18().to(device)
            optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
            criterion = nn.CrossEntropyLoss()
            idx_slice = 0
            for slice in slices:
                idx_slice += 1
                train_loader = torch.utils.data.DataLoader(slice, batch_size=64, shuffle=True)
                for epoch in range(self.epochs):
                    for inputs, labels in train_loader:
                        inputs, labels = inputs.to(device), labels.to(device)
                        #print(k)
                        k += 64
                        optimizer.zero_grad()
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        loss.backward()
                        optimizer.step()
                    print("@ shard:", idx_shard, " @ slice:", idx_slice, "@ epoch:", epoch, "  loss=", loss.item())
                # Save the model after training each slice
                model_path = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice}.pth')
                torch.save(model.state_dict(), model_path)
                self.slice_models[(idx_shard, idx_slice)] = model_path
            self.models.append(model)
        self.training_time = time.time() - start_time  # Record training time





    def load_models(self):
        self.models = []
        self.slice_models = {}
        for idx_shard in range(1, self.S + 1):
            for idx_slice in range(1, self.R + 1):
                model = ModifiedResNet18().to(device)
                model_path = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice}.pth')
                self.slice_models[(idx_shard, idx_slice)] = model_path
                if idx_slice == self.R:
                  print("loading shard:",idx_shard)
                  model.load_state_dict(torch.load(model_path))
                  self.models.append(model)


    def zip_models(self):
        zip_filename = f'{self.model_dir}.zip'
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for root, dirs, files in os.walk(self.model_dir):
                for file in files:
                    zipf.write(os.path.join(root, file), arcname=file)
        return zip_filename

    def upload_models_to_drive(self):
        zip_filename = self.zip_models()
        file = drive.CreateFile({'title': os.path.basename(zip_filename)})
        file.SetContentFile(zip_filename)
        file.Upload()
        print(f'Uploaded file with ID {file.get("id")}')
        return file.get("id")

    def download_and_extract_models_from_drive(self, file_id):
        file = drive.CreateFile({'id': file_id})
        zip_filename = f'{self.model_dir}.zip'
        file.GetContentFile(zip_filename)
        print(f"Downloaded {zip_filename} from Google Drive")
        self.extract_and_load_model(zip_filename)


    def extract_and_load_model(self, zip_filename):
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(self.model_dir)
        self.load_models()






    def predict_for_all_models(self, test_loader):
        #print()
        #print("in predict_for_all_models:")
        #print("test_loader:")
        #print(test_loader)
        all_predictions = []
        for model in self.models:
            model.eval()  # Set model to evaluation mode
            model_predictions = []
            for inputs, _ in test_loader:
                #print("input:",input)
                inputs = inputs.to(device)
                # Get the predictions from the current model
                with torch.no_grad():
                    pred = model(inputs)
                    #print(pred)
                model_predictions.append(pred.cpu())
            all_predictions.append(torch.cat(model_predictions))
        #print("all_predictions:")
        #print(all_predictions)
        #print("_____")
        return all_predictions

    def aggregate_majority_vote(self, test_loader):
        all_predictions = self.predict_for_all_models(test_loader)
        #print("aggregate_majority_vote")
        #print(all_predictions)
        # Aggregate the predictions using a majority vote
        stacked_predictions = torch.stack([torch.argmax(pred, dim=1) for pred in all_predictions])
        aggregated_predictions = torch.mode(stacked_predictions, dim=0)[0]

        prob_array = torch.zeros(10, stacked_predictions.shape[1])
        #print(all_predictions)
        #print(stacked_predictions)

        for col in range(stacked_predictions.shape[1]):
            for num in range(10):
                prob_array[num, col] = torch.mean((stacked_predictions[:, col] == num).float())

        #print(prob_array)

        return aggregated_predictions, prob_array.T

    def aggregate_weighted_average(self, test_loader, validation_loader):
        all_predictions = self.predict_for_all_models(test_loader)
        model_accuracies = self.compute_model_accuracies(validation_loader)
        weighted_predictions = torch.zeros_like(all_predictions[0])
        for model_pred, accuracy in zip(all_predictions, model_accuracies):
            weighted_predictions += model_pred * accuracy
        weighted_predictions = F.softmax(weighted_predictions, dim=1)
        aggregated_predictions = torch.argmax(weighted_predictions, dim=1)
        #print(weighted_predictions)
        return aggregated_predictions, weighted_predictions

    def compute_model_accuracies(self, validation_loader):
        accuracies = []
        for model in self.models:
            correct = 0
            total = 0
            with torch.no_grad():
                for inputs, labels in validation_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            accuracy = correct / total
            accuracies.append(accuracy)
        return torch.tensor(accuracies)

    def aggregate_simple_average(self, test_loader):
        all_predictions = self.predict_for_all_models(test_loader)
        # Aggregate the predictions by simple averaging of probabilities
        averaged_probabilities = torch.mean(torch.stack(all_predictions), dim=0)
        aggregated_predictions = torch.argmax(averaged_probabilities, dim=1)
        #print(averaged_probabilities)
        averaged_probabilities = F.softmax(averaged_probabilities, dim=1)
        return aggregated_predictions, averaged_probabilities

    def evaluate(self, test_loader, aggregation_function):
        aggregated_predictions, all_probs = aggregation_function(test_loader)
        #print(aggregated_predictions)
        all_labels = []
        with torch.no_grad():
            for _, labels in test_loader:
                all_labels.extend(labels.numpy())
        all_preds = aggregated_predictions.cpu().numpy().tolist()
        all_probs = all_probs.cpu().numpy().tolist()
        all_labels = np.array(all_labels).tolist()
        print("all_preds=", all_preds)
        print("all_labels", all_labels)
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')
        auroc = roc_auc_score(all_labels, all_probs, multi_class='ovr')

        return accuracy, f1, precision, recall, auroc

    def evaluate_with_weighted_aggregate(self, test_loader, aggregation_function, validation_loader):
        aggregated_predictions, all_probs = aggregation_function(test_loader, validation_loader)
        all_labels = []
        with torch.no_grad():
            for _, labels in test_loader:
                all_labels.extend(labels.numpy())
        all_preds = aggregated_predictions.cpu().numpy().tolist()
        all_probs = all_probs.cpu().numpy().tolist()
        all_labels = np.array(all_labels).tolist()
        print("all_preds=", all_preds)
        print("all_labels", all_labels)
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds, average='weighted')
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')
        auroc = roc_auc_score(all_labels, all_probs, multi_class='ovr')

        return accuracy, f1, precision, recall, auroc

    # Unlearning Algorithm
    def unlearn(self, dataset, num_samples=500):
        start_time = time.time()  # Start timing
        # Randomly choose samples to forget
        forget_indices = np.random.choice(len(dataset), num_samples, replace=False)
        #print(forget_indices)
        num_retrain_slices = 0


        # Reinitialize models only for affected slices
        shards = self.shard_data(dataset)
        for idx_shard, shard in enumerate(shards, start=1):
            slices = self.slice_data(shard)
            flag = False
            for idx_slice, slice in enumerate(slices, start=1):
                slice_start_idx = (idx_shard - 1) * len(dataset) // self.S
                slice_end_idx = slice_start_idx + idx_slice * len(shard) // self.R
                slice_indices = set(range(slice_start_idx, slice_end_idx))

                print(slice_start_idx," ---> ",slice_end_idx)
                overlap = slice_indices.intersection(forget_indices)
                if overlap:
                    difference = {idx - slice_start_idx for idx in overlap}
                    print("overlap: ", overlap)
                    #print("difference: ", difference)
                    num_retrain_slices += 1
                    flag = True
                    model = ModifiedResNet18().to(device)
                    if idx_slice != 1:  # Avoid loading for the first slice
                        model_path_prepart = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice - 1}.pth')
                        model.load_state_dict(torch.load(model_path_prepart))
                    # Create new slice data without the forgotten indices
                    retained_indices = list(set(range(len(slice))) - difference)
                    retained_set = Subset(slice, retained_indices)
                    retained_loader = DataLoader(retained_set, batch_size=64, shuffle=True)
                    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
                    criterion = nn.CrossEntropyLoss()
                    model.train()
                    for epoch in range(self.epochs):
                        for inputs, labels in retained_loader:
                            inputs, labels = inputs.to(device), labels.to(device)
                            optimizer.zero_grad()
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                            loss.backward()
                            optimizer.step()
                        print(f"@ shard: {idx_shard}, @ slice: {idx_slice}, @ epoch: {epoch + 1}, loss: {loss.item()}")

                    # Save the updated model
                    model_path = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice}.pth')
                    if os.path.exists(model_path):
                        os.remove(model_path)
                    torch.save(model.state_dict(), model_path)
                    self.slice_models[(idx_shard, idx_slice)] = model_path

            if flag == False:
                  print("shard:",idx_shard," model dont change")

        self.unlearning_time = time.time() - start_time  # Record unlearning time
        self.load_models()
        self.unlearning_percent = num_retrain_slices / (self.S * self.R)


    # Unlearning Algorithm
    def unlearn_with_indices(self, dataset, forget_indices):
        start_time = time.time()  # Start timing
        print(forget_indices)
        num_retrain_slices = 0


        # Reinitialize models only for affected slices
        shards = self.shard_data(dataset)
        for idx_shard, shard in enumerate(shards, start=1):
            slices = self.slice_data(shard)
            flag = False
            for idx_slice, slice in enumerate(slices, start=1):
                slice_start_idx = (idx_shard - 1) * len(dataset) // self.S
                slice_end_idx = slice_start_idx + idx_slice * len(shard) // self.R
                slice_indices = set(range(slice_start_idx, slice_end_idx))

                print(slice_start_idx," ---> ",slice_end_idx)
                overlap = slice_indices.intersection(forget_indices)
                if overlap:
                    difference = {idx - slice_start_idx for idx in overlap}
                    print("overlap: ", overlap)
                    #print("difference: ", difference)
                    num_retrain_slices += 1
                    flag = True
                    model = ModifiedResNet18().to(device)
                    if idx_slice != 1:  # Avoid loading for the first slice
                        print("idx_slice != 1")
                        model_path_prepart = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice - 1}.pth')
                        model.load_state_dict(torch.load(model_path_prepart))
                    # Create new slice data without the forgotten indices
                    retained_indices = list(set(range(len(slice))) - difference)
                    retained_set = Subset(slice, retained_indices)
                    retained_loader = DataLoader(retained_set, batch_size=64, shuffle=True)
                    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
                    criterion = nn.CrossEntropyLoss()
                    model.train()
                    for epoch in range(self.epochs):
                        for inputs, labels in retained_loader:
                            inputs, labels = inputs.to(device), labels.to(device)
                            optimizer.zero_grad()
                            outputs = model(inputs)
                            loss = criterion(outputs, labels)
                            loss.backward()
                            optimizer.step()
                        print(f"@ shard: {idx_shard}, @ slice: {idx_slice}, @ epoch: {epoch + 1}, loss: {loss.item()}")

                    # Save the updated model
                    model_path = os.path.join(self.model_dir, f'model_shard{idx_shard}_slice{idx_slice}.pth')
                    if os.path.exists(model_path):
                        os.remove(model_path)
                    torch.save(model.state_dict(), model_path)
                    self.slice_models[(idx_shard, idx_slice)] = model_path

            if flag == False:
                  print("shard:",idx_shard," model dont change")

        self.unlearning_time = time.time() - start_time  # Record unlearning time
        self.load_models()
        self.unlearning_percent = num_retrain_slices / (self.S * self.R)


    def calculate_losses(self, data_loader):
        all_losses = []
        all_predictions = self.predict_for_all_models(data_loader)
        # Aggregate the predictions by simple averaging of probabilities
        averaged_probabilities = torch.mean(torch.stack(all_predictions), dim=0).to(device)

        # Collect all labels
        all_labels = []
        for _, labels in data_loader:
            all_labels.extend(labels.numpy())

        # Convert labels to tensor
        all_labels = torch.tensor(all_labels, dtype=torch.long).to(device)

        # Calculate loss for all samples at once
        with torch.no_grad():
            loss = F.cross_entropy(averaged_probabilities, all_labels, reduction='none')
            all_losses.extend(loss.cpu().numpy())

        return np.array(all_losses)



# **Simulation Question 1.**

Implement the learning phase of the algorithm, and select the based
models to be pretrained ResNet 18(you will need to add some fully connected layer(s) in
order to make it suitable for the given dataset!). Use S = 5, 10, 20 and R = 5, 10, 20 and train
different models on your data in each case, and report F1-score, accuracy, precision, recall and
AUROC of all of your models with all of your proposed aggregation methods.


**Trainig Data:**

In [ ]:
# transform to include resizing to 224x224
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [ ]:
cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:05<00:00, 30231347.49it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


## **S=5 R=5 :**

**Initialize SISA**

In [ ]:
S = 5
R = 5
epochs = 5

sisa_S5_R5 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1YQwzzpRoQuNQ1ubKp5LDbthBdSnMVIoP'

# Download and extract models
sisa_S5_R5.download_and_extract_models_from_drive(file_id)

**load model from zip file:**

In [ ]:
sisa_S5_R5.extract_and_load_model('/content/models_S5_R5.zip')

loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


**Training:**

In [ ]:
sisa_S5_R5.train(cifar10_train)

**Upload model on google drive:**

In [ ]:
sisa_S5_R5.upload_models_to_drive()

Uploaded file with ID 1YQwzzpRoQuNQ1ubKp5LDbthBdSnMVIoP


'1YQwzzpRoQuNQ1ubKp5LDbthBdSnMVIoP'

**Test Data:**

In [ ]:
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:
# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S5_R5.evaluate_with_weighted_aggregate(test_loader, sisa_S5_R5.aggregate_weighted_average, validation_loader)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 8, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 8, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 0, 4, 5, 6, 5, 1, 5, 8, 8, 0, 4, 0, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 4, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9276, F1-score: 0.9272, Precision: 0.9277, Recall: 0.9276, AUROC: 0.9961


*  **average aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R5.evaluate(test_loader, sisa_S5_R5.aggregate_simple_average)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 8, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 8, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 0, 4, 5, 6, 5, 1, 5, 8, 8, 0, 4, 0, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 4, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9275, F1-score: 0.9271, Precision: 0.9277, Recall: 0.9275, AUROC: 0.9966


*  **majority vote aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R5.evaluate(test_loader, sisa_S5_R5.aggregate_majority_vote)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 8, 4, 9, 9, 5, 0, 6, 5, 6, 0, 9, 3, 8, 7, 6, 0, 8, 0, 3, 8, 8, 7, 7, 4, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 4, 9, 3, 4, 5, 6, 5, 1, 5, 8, 8, 0, 4, 8, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 2, 4, 3, 9, 0, 6, 1, 0, 9, 1, 8, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 9, 2, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9213, F1-score: 0.9210, Precision: 0.9216, Recall: 0.9213, AUROC: 0.9856


## **S=5 R=10 :**

**Initialize SISA**

In [ ]:
S = 5
R = 10
epochs = 5

sisa_S5_R10 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1e8BzxOKJI5ZkZiuVOcuLcXwio06lWnRk'

# Download and extract models
sisa_S5_R10.download_and_extract_models_from_drive(file_id)

**load model from zip file:**

In [ ]:
sisa_S5_R10.extract_and_load_model('/content/models_S5_R10.zip')

**Training:**

In [ ]:
sisa_S5_R10.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 1.844420075416565
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 1.0398776531219482
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 0.6583490967750549
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 0.1887321025133133
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 0.13890446722507477
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 0.5308066606521606
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 0.3860238492488861
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 0.09960657358169556
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 0.01327322144061327
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.02511678636074066
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 0.4546278417110443
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.0650818720459938
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.1344113051891327
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.010339693166315556
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.011672000400722027
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 0.5218769907951355
@ shard: 1  @ sli

**Upload model to google drive:**

In [ ]:
sisa_S5_R10.upload_models_to_drive()

Uploaded file with ID 1e8BzxOKJI5ZkZiuVOcuLcXwio06lWnRk


'1e8BzxOKJI5ZkZiuVOcuLcXwio06lWnRk'

**Test Data:**

In [ ]:
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:
# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S5_R10.evaluate_with_weighted_aggregate(test_loader, sisa_S5_R10.aggregate_weighted_average, validation_loader)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 8, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 8, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 5, 3, 7, 5, 6, 3, 6, 2, 1, 2, 6, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 6, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 0, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 0, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 5, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 0, 4, 5, 6, 5, 1, 3, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 2, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9286, F1-score: 0.9284, Precision: 0.9286, Recall: 0.9286, AUROC: 0.9963


*  **average aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R10.evaluate(test_loader, sisa_S5_R10.aggregate_simple_average)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 8, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 8, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 5, 3, 7, 5, 6, 3, 6, 2, 1, 2, 6, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 6, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 0, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 0, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 5, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 0, 4, 5, 6, 5, 1, 3, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 2, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9286, F1-score: 0.9284, Precision: 0.9286, Recall: 0.9286, AUROC: 0.9966


*  **majority vote aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R10.evaluate(test_loader, sisa_S5_R10.aggregate_majority_vote)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 0, 6, 5, 6, 0, 9, 3, 0, 7, 6, 9, 8, 7, 3, 8, 8, 7, 6, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 6, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 2, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 0, 1, 3, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 0, 4, 7, 6, 5, 1, 3, 8, 8, 0, 4, 7, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 2, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 4, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9176, F1-score: 0.9176, Precision: 0.9181, Recall: 0.9176, AUROC: 0.9861


## **S=5 R=20 :**

**Initialize SISA**

In [ ]:
S = 5
R = 20
epochs = 5

sisa_S5_R20 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1oksdaxjVrjOjAtNcbxRvNhxvgWEeBLPe'

sisa_S5_R20.download_and_extract_models_from_drive(file_id)

**load model from zip file:**

In [ ]:
sisa_S5_R20.extract_and_load_model('/content/models_S5_R20.zip')

**Training:**

In [ ]:
sisa_S5_R20.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 2.1482303142547607
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 1.8220659494400024
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 1.0084840059280396
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 0.573268711566925
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 0.3506477177143097
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 0.6048738956451416
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 0.42601698637008667
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 0.24793308973312378
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 0.08550507575273514
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.05216291919350624
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 0.34708499908447266
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.17308472096920013
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.06940431892871857
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.015653224661946297
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.012534989975392818
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 0.7191556692123413
@ shard: 1  @ 

**Upload model to google drive:**

In [ ]:
sisa_S5_R20.upload_models_to_drive()

Uploaded file with ID 1oksdaxjVrjOjAtNcbxRvNhxvgWEeBLPe


'1oksdaxjVrjOjAtNcbxRvNhxvgWEeBLPe'

**Test Data:**

In [ ]:
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:
# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S5_R20.evaluate_with_weighted_aggregate(test_loader, sisa_S5_R20.aggregate_weighted_average, validation_loader)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 3, 5, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 3, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 2, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 3, 9, 0, 4, 5, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 9, 2, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9202, F1-score: 0.9202, Precision: 0.9206, Recall: 0.9202, AUROC: 0.9957


*  **average aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R20.evaluate(test_loader, sisa_S5_R20.aggregate_simple_average)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 3, 5, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 3, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 2, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 3, 9, 0, 4, 5, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 9, 2, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9202, F1-score: 0.9202, Precision: 0.9207, Recall: 0.9202, AUROC: 0.9960


*  **majority vote aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R20.evaluate(test_loader, sisa_S5_R20.aggregate_majority_vote)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 7, 6, 0, 9, 3, 0, 7, 6, 9, 8, 9, 3, 8, 8, 7, 3, 5, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 5, 9, 4, 3, 9, 0, 4, 3, 6, 5, 1, 3, 8, 8, 0, 4, 7, 5, 5, 1, 1, 8, 9, 0, 0, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 0, 2, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 9, 2, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.7f}, F1-score: {f1:.7f}, Precision: {precision:.7f}, Recall: {recall:.7f}, AUROC: {auroc:.7f}')

Accuracy: 0.9139000, F1-score: 0.9140602, Precision: 0.9148205, Recall: 0.9139000, AUROC: 0.9845573


## **S=10 R=5 :**

**Initialize SISA**

In [ ]:
S = 10
R = 5
epochs = 5

sisa_S10_R5 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1muwLWjXoCIQiVcwq5F3aqZEo6kuriMei'

# Download and extract models
sisa_S10_R5.download_and_extract_models_from_drive(file_id)


**load model from zip file:**

In [ ]:
sisa_S10_R5.extract_and_load_model('/content/models_S10_R5.zip')

**Training:**

In [ ]:
sisa_S10_R5.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 155MB/s]


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 1.754080057144165
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 0.9414708018302917
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 0.5474135279655457
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 0.24070587754249573
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 0.08914732933044434
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 0.8012474179267883
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 0.31070682406425476
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 0.05554519221186638
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 0.06831629574298859
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.006657636724412441
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 1.0702091455459595
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.21707050502300262
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.05158375948667526
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.02590019442141056
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.01872529834508896
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 0.82098788022995
@ shard: 1  @ sl

**Upload model to google drive:**

In [ ]:
sisa_S10_R5.upload_models_to_drive()

Uploaded file with ID 1muwLWjXoCIQiVcwq5F3aqZEo6kuriMei


'1muwLWjXoCIQiVcwq5F3aqZEo6kuriMei'

**Test Data:**

In [ ]:
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S10_R5.evaluate_with_weighted_aggregate(test_loader, sisa_S10_R5.aggregate_weighted_average, validation_loader)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 5, 9, 4, 4, 9, 0, 4, 7, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 9, 4, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9237, F1-score: 0.9234, Precision: 0.9238, Recall: 0.9237, AUROC: 0.9951



*  **average aggregation:**

In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S10_R5.evaluate(test_loader, sisa_S10_R5.aggregate_simple_average)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 5, 9, 4, 4, 9, 0, 4, 7, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 9, 4, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9238, F1-score: 0.9235, Precision: 0.9239, Recall: 0.9238, AUROC: 0.9961


*  **majority vote aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S10_R5.evaluate(test_loader, sisa_S10_R5.aggregate_majority_vote)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 0, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 4, 9, 0, 4, 7, 6, 5, 1, 7, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 0, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 4, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9198, F1-score: 0.9196, Precision: 0.9200, Recall: 0.9198, AUROC: 0.9896


## **S=10 R=10 :**


**Initialize SISA**

In [ ]:
S = 10
R = 10
epochs = 5

sisa_S10_R10 = SISA(S, R, epochs)


**download model from saved files:**

In [ ]:

# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1dCWYPHZbQHD81XLNB82aaNoQxoCVGNBG'

# Download and extract models
sisa_S10_R10.download_and_extract_models_from_drive(file_id)


Downloaded models_S10_R10.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 150MB/s]


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5
loading shard: 6
loading shard: 7
loading shard: 8
loading shard: 9
loading shard: 10


**load model from zip file:**

In [ ]:
sisa_S10_R10.extract_and_load_model('/content/models_S5_R10.zip')

**Training:**

In [ ]:
sisa_S10_R10.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 2.197660446166992
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 1.8291587829589844
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 1.285563349723816
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 0.6856483221054077
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 0.4026287794113159
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 0.8024006485939026
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 0.4799879789352417
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 0.2295444756746292
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 0.08417122066020966
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.02464919537305832
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 0.4402303099632263
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.21059870719909668
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.14852440357208252
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.06747865676879883
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.038607046008110046
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 0.30734989047050476
@ shard: 1  @ slic


**Upload model to google drive:**

In [ ]:
sisa_S10_R10.upload_models_to_drive()

Uploaded file with ID 1dCWYPHZbQHD81XLNB82aaNoQxoCVGNBG


'1dCWYPHZbQHD81XLNB82aaNoQxoCVGNBG'

**Test Data:**

In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)


Files already downloaded and verified



*  **weighted average aggregation:**

In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S10_R10.evaluate_with_weighted_aggregate(test_loader, sisa_S10_R10.aggregate_weighted_average, validation_loader)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 0, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 3, 5, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 2, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 6, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 5, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 5, 9, 4, 4, 9, 0, 4, 5, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 9, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 8, 2, 1, 4, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9162, F1-score: 0.9161, Precision: 0.9163, Recall: 0.9162, AUROC: 0.9948


*  **average aggregation:**

In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S10_R10.evaluate(test_loader, sisa_S10_R10.aggregate_simple_average)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 0, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 3, 5, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 2, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 6, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 5, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 5, 9, 4, 4, 9, 0, 4, 5, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 9, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 8, 2, 1, 4, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9161, F1-score: 0.9160, Precision: 0.9162, Recall: 0.9161, AUROC: 0.9957


*  **majority vote aggregation:**

In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S10_R10.evaluate(test_loader, sisa_S10_R10.aggregate_majority_vote)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 5, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 6, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 2, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 6, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 5, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 5, 9, 4, 4, 9, 0, 4, 5, 6, 5, 1, 7, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 0, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 2, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9135, F1-score: 0.9134, Precision: 0.9138, Recall: 0.9135, AUROC: 0.9896


## **S=10 R=20 :**

**Initialize SISA**

In [ ]:

S = 10
R = 20
epochs = 5

sisa_S10_R20 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1x0MfTN5ztDCumz2nBiUgnRvG444v2TUp'

# Download and extract models
sisa_S10_R20.download_and_extract_models_from_drive(file_id)



**load model from zip file:**

In [ ]:
sisa_S10_R20.extract_and_load_model('/content/models_S5_R10.zip')

**Training:**

In [ ]:
sisa_S10_R20.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 215MB/s]


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 2.308741807937622
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 2.1161305904388428
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 1.7311755418777466
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 1.2957756519317627
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 0.7803904414176941
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 1.2502886056900024
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 0.6303051114082336
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 0.40171483159065247
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 0.2922915518283844
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.20430734753608704
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 0.8749842643737793
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.5437781810760498
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.13429467380046844
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.1224508285522461
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.04768079146742821
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 0.8049405217170715
@ shard: 1  @ slice: 

**Upload model to google drive:**

In [ ]:
sisa_S10_R20.upload_models_to_drive()

Uploaded file with ID 1x0MfTN5ztDCumz2nBiUgnRvG444v2TUp


'1x0MfTN5ztDCumz2nBiUgnRvG444v2TUp'

**Test Data:**

In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)


Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S10_R20.evaluate_with_weighted_aggregate(test_loader, sisa_S10_R20.aggregate_weighted_average, validation_loader)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 3, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 8, 7, 6, 9, 8, 0, 3, 8, 8, 7, 3, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 3, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 3, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 5, 8, 3, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 3, 9, 3, 4, 3, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 8, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 9, 2, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')



Accuracy: 0.9060, F1-score: 0.9064, Precision: 0.9080, Recall: 0.9060, AUROC: 0.9939


*  **average aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S10_R20.evaluate(test_loader, sisa_S10_R20.aggregate_simple_average)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 3, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 8, 7, 6, 9, 8, 0, 3, 8, 8, 7, 3, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 3, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 3, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 5, 8, 3, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 3, 9, 3, 4, 3, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 8, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 9, 2, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9060, F1-score: 0.9064, Precision: 0.9081, Recall: 0.9060, AUROC: 0.9947



*  **majority vote aggregation:**


In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S10_R20.evaluate(test_loader, sisa_S10_R20.aggregate_majority_vote)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 3, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 8, 7, 6, 0, 8, 0, 3, 8, 8, 7, 3, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 2, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 3, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 5, 8, 3, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 4, 9, 0, 4, 3, 6, 5, 1, 7, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 0, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 0, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 2, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.8987, F1-score: 0.8991, Precision: 0.9008, Recall: 0.8987, AUROC: 0.9883


## **S=20 R=5 :**

**Initialize SISA**

In [ ]:
S = 20
R = 5
epochs = 5

sisa_S20_R5 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:

# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1DZjk7EaauGZNq3H_Un-OViFw5cCiNQAS'

# Download and extract models
sisa_S5_R5.download_and_extract_models_from_drive(file_id)




**load model from zip file:**

In [ ]:
sisa_S20_R5.extract_and_load_model('/content/models_S20_R5.zip')



**Training:**

In [ ]:
sisa_S20_R5.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 123MB/s]


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 2.2177443504333496
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 1.8043274879455566
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 1.157824993133545
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 0.47671324014663696
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 0.3299572169780731
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 0.895873486995697
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 0.45369789004325867
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 0.062271520495414734
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 0.03745722025632858
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.05470509082078934
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 0.5892884731292725
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.24225296080112457
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.21179834008216858
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.04956052079796791
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.032162752002477646
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 0.7030489444732666
@ shard: 1  @ s

**Upload model to google drive:**

In [ ]:
sisa_S20_R5.upload_models_to_drive()

Uploaded file with ID 1DZjk7EaauGZNq3H_Un-OViFw5cCiNQAS


'1DZjk7EaauGZNq3H_Un-OViFw5cCiNQAS'

**Test Data:**

In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)


Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S20_R5.evaluate_with_weighted_aggregate(test_loader, sisa_S20_R5.aggregate_weighted_average, validation_loader)


all_preds= [2, 4, 6, 9, 4, 6, 8, 7, 9, 6, 6, 7, 3, 4, 4, 1, 0, 8, 0, 7, 0, 4, 3, 7, 4, 1, 0, 1, 0, 0, 0, 4, 1, 7, 4, 9, 7, 4, 1, 2, 5, 3, 2, 1, 5, 2, 5, 1, 7, 9, 1, 9, 8, 9, 5, 4, 9, 5, 9, 2, 0, 2, 5, 5, 2, 4, 0, 6, 6, 6, 5, 0, 2, 7, 0, 0, 2, 7, 7, 6, 2, 0, 7, 2, 5, 5, 5, 8, 7, 8, 0, 9, 8, 2, 3, 4, 6, 6, 6, 8, 9, 7, 0, 6, 2, 0, 5, 6, 0, 8, 1, 1, 9, 8, 6, 2, 3, 4, 7, 1, 3, 9, 1, 0, 8, 6, 7, 3, 4, 5, 8, 9, 1, 2, 9, 0, 6, 0, 7, 4, 9, 7, 9, 5, 8, 1, 6, 3, 1, 9, 8, 9, 6, 6, 4, 7, 8, 0, 0, 5, 6, 5, 3, 9, 7, 4, 5, 7, 1, 7, 5, 3, 7, 0, 8, 3, 5, 5, 5, 1, 6, 5, 9, 7, 9, 2, 3, 7, 4, 3, 0, 9, 5, 7, 3, 7, 7, 2, 7, 1, 7, 0, 7, 4, 5, 7, 9, 2, 2, 4, 6, 3, 6, 0, 4, 2, 5, 5, 2, 2, 4, 4, 2, 2, 7, 0, 2, 6, 8, 3, 6, 2, 7, 0, 6, 8, 0, 9, 8, 6, 4, 7, 2, 7, 6, 6, 1, 0, 8, 9, 0, 4, 6, 7, 3, 3, 6, 7, 4, 2, 1, 5, 5, 6, 2, 4, 2, 2, 7, 1, 5, 2, 8, 4, 0, 9, 6, 2, 9, 0, 1, 2, 6, 8, 4, 5, 3, 5, 5, 2, 7, 3, 3, 5, 4, 8, 8, 1, 1, 0, 6, 4, 2, 3, 2, 3, 1, 7, 6, 1, 9, 4, 3, 0, 5, 8, 7, 1, 7, 3, 5, 3, 9, 7, 3, 2, 1, 3, 0, 2

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.8970, F1-score: 0.8968, Precision: 0.8982, Recall: 0.8970, AUROC: 0.9916



*  **average aggregation:**


In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S20_R5.evaluate(test_loader, sisa_S20_R5.aggregate_simple_average)


all_preds= [2, 4, 6, 9, 4, 6, 8, 7, 9, 6, 6, 7, 3, 4, 4, 1, 0, 8, 0, 7, 0, 4, 3, 7, 4, 1, 0, 1, 0, 0, 0, 4, 1, 7, 4, 9, 7, 4, 1, 2, 5, 3, 2, 1, 5, 2, 5, 1, 7, 9, 1, 9, 8, 9, 5, 4, 9, 5, 9, 2, 0, 2, 5, 5, 2, 4, 0, 6, 6, 6, 5, 0, 2, 7, 0, 0, 2, 7, 7, 6, 2, 0, 7, 2, 5, 5, 5, 8, 7, 8, 0, 9, 8, 2, 3, 4, 6, 6, 6, 8, 9, 7, 0, 6, 2, 0, 5, 6, 0, 8, 1, 1, 9, 8, 6, 2, 3, 4, 7, 1, 3, 9, 1, 0, 8, 6, 7, 3, 4, 5, 8, 9, 1, 2, 9, 0, 6, 0, 7, 4, 9, 7, 9, 5, 8, 1, 6, 3, 1, 9, 8, 9, 6, 6, 4, 7, 8, 0, 0, 5, 6, 5, 3, 9, 7, 4, 5, 7, 1, 7, 5, 3, 7, 0, 8, 3, 5, 5, 5, 1, 6, 5, 9, 7, 9, 2, 3, 7, 4, 3, 0, 9, 5, 7, 3, 7, 7, 2, 7, 1, 7, 0, 7, 4, 5, 7, 9, 2, 2, 4, 6, 3, 6, 0, 4, 2, 5, 5, 2, 2, 4, 4, 2, 2, 7, 0, 2, 6, 8, 3, 6, 2, 7, 0, 6, 8, 0, 9, 8, 6, 4, 7, 2, 7, 6, 6, 1, 0, 8, 9, 0, 4, 6, 7, 3, 3, 6, 7, 4, 2, 1, 5, 5, 6, 2, 4, 2, 2, 7, 1, 5, 2, 8, 4, 0, 9, 6, 2, 9, 0, 1, 2, 6, 8, 4, 5, 3, 5, 5, 2, 7, 3, 3, 5, 4, 8, 8, 1, 1, 0, 6, 4, 2, 3, 2, 3, 1, 7, 6, 1, 9, 4, 3, 0, 5, 8, 7, 1, 7, 3, 5, 3, 9, 7, 3, 2, 1, 3, 0, 2

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.8970, F1-score: 0.8968, Precision: 0.8982, Recall: 0.8970, AUROC: 0.9936



*  **majority vote aggregation:**


In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S20_R5.evaluate(test_loader, sisa_S20_R5.aggregate_majority_vote)


all_preds= [2, 4, 6, 9, 4, 6, 8, 7, 9, 6, 6, 7, 3, 4, 4, 1, 0, 8, 0, 7, 0, 4, 3, 7, 4, 1, 0, 1, 0, 0, 0, 4, 1, 7, 4, 9, 7, 4, 1, 2, 5, 3, 2, 1, 5, 2, 5, 1, 7, 9, 1, 9, 8, 9, 5, 4, 9, 5, 9, 2, 0, 2, 2, 5, 2, 4, 0, 6, 6, 5, 5, 0, 2, 7, 0, 0, 2, 7, 7, 6, 2, 0, 7, 2, 5, 5, 5, 8, 7, 8, 0, 9, 8, 2, 3, 4, 6, 6, 6, 8, 9, 7, 0, 6, 2, 0, 5, 6, 0, 8, 1, 1, 9, 8, 6, 2, 3, 4, 7, 1, 3, 9, 1, 0, 8, 6, 7, 6, 4, 5, 8, 9, 1, 2, 9, 0, 6, 0, 7, 4, 9, 7, 9, 5, 8, 1, 6, 3, 1, 9, 8, 9, 6, 6, 4, 7, 8, 0, 0, 5, 6, 5, 3, 9, 7, 4, 5, 3, 1, 7, 5, 3, 7, 0, 8, 3, 5, 5, 5, 1, 6, 5, 9, 7, 9, 2, 3, 7, 4, 3, 0, 9, 5, 7, 3, 7, 7, 2, 7, 1, 7, 0, 7, 4, 5, 7, 9, 2, 2, 4, 6, 3, 6, 0, 4, 2, 5, 5, 2, 2, 4, 4, 2, 2, 7, 0, 2, 6, 8, 3, 6, 2, 7, 0, 6, 8, 0, 9, 8, 6, 4, 7, 2, 7, 6, 6, 1, 0, 8, 9, 0, 4, 6, 7, 3, 3, 6, 7, 4, 2, 1, 5, 5, 6, 2, 4, 2, 2, 7, 1, 5, 2, 8, 4, 0, 9, 6, 2, 9, 0, 1, 2, 6, 8, 4, 5, 3, 5, 5, 2, 7, 3, 3, 5, 4, 8, 8, 1, 1, 0, 6, 4, 2, 3, 2, 3, 1, 7, 6, 1, 9, 4, 3, 0, 5, 8, 7, 1, 7, 3, 5, 3, 9, 7, 3, 2, 1, 3, 8, 2

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.8910, F1-score: 0.8907, Precision: 0.8912, Recall: 0.8910, AUROC: 0.9884


## **S=20 R=10 :**


**Initialize SISA**

In [ ]:
S = 20
R = 10
epochs = 5

sisa_S20_R10 = SISA(S, R, epochs)


**download model from saved files:**

In [ ]:

# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1wMXld7nzoqE2LDD2Vts3EVLc-GVaNvzS'

# Download and extract models
sisa_S5_R5.download_and_extract_models_from_drive(file_id)



**load model from zip file:**

In [ ]:
sisa_S20_R10.extract_and_load_model('/content/models_S20_R10.zip')


**Training:**

In [ ]:
sisa_S20_R10.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 2.287726879119873
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 2.14151668548584
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 1.7443186044692993
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 1.348589539527893
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 0.8728532195091248
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 1.118328332901001
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 0.7533826231956482
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 0.319953054189682
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 0.12623508274555206
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.07535399496555328
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 0.809587299823761
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.6274542808532715
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.2668731212615967
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.11613112688064575
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.053472716361284256
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 0.6874566078186035
@ shard: 1  @ slice: 4 @ e

**Upload model to google drive:**

In [ ]:
sisa_S20_R10.upload_models_to_drive()

Uploaded file with ID 1wMXld7nzoqE2LDD2Vts3EVLc-GVaNvzS


'1wMXld7nzoqE2LDD2Vts3EVLc-GVaNvzS'

**Test Data:**

In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)


Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S20_R10.evaluate_with_weighted_aggregate(test_loader, sisa_S20_R10.aggregate_weighted_average, validation_loader)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 0, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 7, 3, 8, 8, 7, 3, 4, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 7, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 2, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 6, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 8, 9, 0, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 0, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 5, 9, 4, 4, 9, 0, 4, 3, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 8, 2, 1, 4, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.8979, F1-score: 0.8976, Precision: 0.8978, Recall: 0.8979, AUROC: 0.9919



*  **average aggregation:**


In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S20_R10.evaluate(test_loader, sisa_S20_R10.aggregate_simple_average)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 0, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 7, 3, 8, 8, 7, 3, 4, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 7, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 6, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 8, 9, 0, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 0, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 5, 9, 4, 4, 9, 0, 4, 3, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 8, 2, 1, 4, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.8982, F1-score: 0.8979, Precision: 0.8982, Recall: 0.8982, AUROC: 0.9936



*  **majority vote aggregation:**


In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S20_R10.evaluate(test_loader, sisa_S20_R10.aggregate_majority_vote)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 3, 9, 3, 4, 9, 9, 5, 0, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 7, 6, 8, 8, 7, 3, 5, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 7, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 3, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 2, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 0, 9, 0, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 0, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 0, 4, 3, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 2, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 9, 2, 1, 4, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.8922, F1-score: 0.8919, Precision: 0.8922, Recall: 0.8922, AUROC: 0.9901


## **S=20 R=20 :**

**Initialize SISA**

In [ ]:
S = 20
R = 20
epochs = 5

sisa_S20_R20 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = ''

# Download and extract models
sisa_S20_R20.download_and_extract_models_from_drive(file_id)

**load model from zip file:**

In [ ]:
sisa_S20_R20.extract_and_load_model('/content/models_S20_R20.zip')

**Training:**

In [ ]:
sisa_S20_R20.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 194MB/s]


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 2.3889825344085693
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 2.2419986724853516
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 2.0042974948883057
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 1.891906976699829
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 1.640162706375122
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 2.067854404449463
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 1.6614176034927368
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 1.3573142290115356
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 1.3034050464630127
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.8389109969139099
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 1.3219335079193115
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.9343261122703552
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.7324256300926208
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.5117673277854919
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.35375502705574036
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 0.8943036198616028
@ shard: 1  @ slice: 4 @ e


**Upload model to google drive:**

In [ ]:
sisa_S20_R20.upload_models_to_drive()

**Test Data:**

In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)


Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:
# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S20_R20.evaluate_with_weighted_aggregate(test_loader, sisa_S20_R20.aggregate_weighted_average, validation_loader)

all_preds= [3, 8, 8, 0, 6, 6, 1, 2, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 3, 9, 3, 4, 9, 9, 5, 0, 6, 7, 6, 0, 9, 3, 9, 7, 6, 9, 8, 7, 3, 8, 8, 7, 3, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 2, 6, 8, 7, 4, 0, 2, 2, 1, 3, 0, 4, 6, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 5, 8, 2, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 8, 9, 0, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 0, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 3, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 4, 9, 3, 4, 3, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 3, 1, 1, 8, 9, 0, 3, 1, 8, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 8, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 8, 2, 9, 2, 8, 6, 0, 2, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 7, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.8812, F1-score: 0.8814, Precision: 0.8839, Recall: 0.8812, AUROC: 0.9902


*  **average aggregation:**

In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S20_R20.evaluate(test_loader, sisa_S20_R20.aggregate_simple_average)


all_preds= [3, 8, 8, 0, 6, 6, 1, 2, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 3, 9, 3, 4, 9, 9, 5, 0, 6, 7, 6, 0, 9, 3, 9, 7, 6, 9, 8, 7, 3, 8, 8, 7, 3, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 2, 2, 1, 3, 0, 4, 6, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 5, 8, 2, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 8, 9, 0, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 0, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 3, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 4, 9, 3, 4, 3, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 3, 1, 1, 8, 9, 0, 3, 1, 8, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 8, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 8, 2, 9, 2, 8, 6, 0, 2, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 7, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.8812, F1-score: 0.8815, Precision: 0.8842, Recall: 0.8812, AUROC: 0.9915



*  **majority vote aggregation:**


In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S20_R20.evaluate(test_loader, sisa_S20_R20.aggregate_majority_vote)

all_preds= [3, 8, 8, 0, 6, 6, 1, 2, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 3, 9, 3, 4, 9, 9, 5, 0, 6, 7, 6, 0, 9, 3, 9, 7, 6, 9, 8, 7, 3, 8, 8, 7, 7, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 2, 6, 8, 7, 4, 0, 2, 2, 1, 3, 0, 4, 6, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 5, 8, 2, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 8, 9, 0, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 3, 8, 5, 0, 8, 7, 1, 3, 0, 5, 7, 9, 3, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 4, 9, 0, 4, 3, 2, 5, 1, 5, 8, 8, 0, 4, 9, 5, 3, 1, 1, 8, 9, 0, 3, 1, 8, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 8, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 8, 2, 9, 2, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.8745, F1-score: 0.8748, Precision: 0.8781, Recall: 0.8745, AUROC: 0.9872


# **Simulation Question 2.**
Implement the unlearning algorithm on all your trained models in the previous question and ”forget” 500 randomly chosen data of the whole dataset and measure the performance using your proposed metric, also report F1-score, accuracy, precision, recall and AUROC of all of your models with all of your proposed aggregation methods.

## **S=5 R=5:**

**Initialize SISA**

In [ ]:
S = 5
R = 5
epochs = 5

sisa_S5_R5 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1YQwzzpRoQuNQ1ubKp5LDbthBdSnMVIoP'

# Download and extract models
sisa_S5_R5.download_and_extract_models_from_drive(file_id)

Downloaded models_S5_R5.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


**load model from zip file:**

In [ ]:
sisa_S5_R5.extract_and_load_model('/content/models_S5_R5.zip')

**Unlearning:**

In [ ]:
sisa_S5_R5.unlearn(cifar10_train, 500)

0  --->  2000
overlap:  {774, 664, 671, 165, 678, 1962, 1326, 1587, 438, 63, 836, 844, 1620, 1749, 1496, 1501, 861, 1375, 221, 1503, 487, 746, 1262, 1016, 1662}
@ shard: 1, @ slice: 1, @ epoch: 1, loss: 0.8478503823280334
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 0.8782046437263489
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 0.26479071378707886
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 0.16672389209270477
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 0.049657534807920456
0  --->  4000
overlap:  {2181, 774, 2440, 3977, 2956, 664, 2458, 671, 2210, 165, 678, 3239, 2729, 1962, 1326, 3760, 1587, 438, 3512, 2489, 2234, 63, 2239, 2114, 836, 3142, 844, 1620, 1749, 1496, 1501, 861, 1375, 221, 1503, 3298, 487, 746, 1262, 2289, 3059, 1016, 1662}
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 0.500704824924469
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 0.14252495765686035
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.05478706955909729
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 0.04449235647916794
@ shard: 1, @ s

**performance using your proposed metric**

In [ ]:
print("unlearning time:",sisa_S5_R5.unlearning_time,"sec")
print("unlearning percent:",sisa_S5_R5.unlearning_percent*100,"%")

unlearning time: 736.6806917190552 sec
unlearning percent: 100.0 %


**Test Data:**

In [ ]:
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

Files already downloaded and verified



*  **weighted average aggregation:**

In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S5_R5.evaluate_with_weighted_aggregate(test_loader, sisa_S5_R5.aggregate_weighted_average, validation_loader)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 3, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 0, 4, 3, 6, 5, 1, 3, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 4, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9298, F1-score: 0.9296, Precision: 0.9300, Recall: 0.9298, AUROC: 0.9961


*  **average aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R5.evaluate(test_loader, sisa_S5_R5.aggregate_simple_average)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 7, 3, 8, 8, 7, 7, 4, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 0, 4, 3, 6, 5, 1, 3, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 4, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9292, F1-score: 0.9290, Precision: 0.9295, Recall: 0.9292, AUROC: 0.9965


*  **majority vote aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R5.evaluate(test_loader, sisa_S5_R5.aggregate_majority_vote)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 3, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 7, 3, 8, 8, 7, 3, 4, 3, 7, 5, 6, 3, 6, 2, 1, 2, 5, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 3, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 8, 6, 5, 6, 3, 8, 2, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 5, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 4, 9, 0, 4, 3, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 1, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 8, 2, 9, 2, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9240, F1-score: 0.9239, Precision: 0.9244, Recall: 0.9240, AUROC: 0.9872


## **S=5 R=10:**

**Initialize SISA**

In [ ]:
S = 5
R = 10
epochs = 5

sisa_S5_R10 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1e8BzxOKJI5ZkZiuVOcuLcXwio06lWnRk'

# Download and extract models
sisa_S5_R10.download_and_extract_models_from_drive(file_id)

Downloaded models_S5_R10.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


**load model from zip file:**

In [ ]:
sisa_S5_R10.extract_and_load_model('/content/models_S5_R10.zip')

**Unlearning:**

In [ ]:
sisa_S5_R10.unlearn(cifar10_train, 500)

0  --->  1000
overlap:  {993, 325, 681, 778, 907, 364, 398, 880, 688, 465, 373}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1, @ slice: 1, @ epoch: 1, loss: 1.7819489240646362
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 0.8801811337471008
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 0.5574976205825806
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 0.22958049178123474
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 0.0797291025519371
0  --->  2000
overlap:  {778, 907, 398, 1428, 681, 1963, 1710, 688, 1585, 1203, 325, 465, 1237, 1499, 993, 1767, 364, 1773, 880, 1136, 373}
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 0.5695614218711853
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 0.24656729400157928
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.3293413817882538
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 0.03639056906104088
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 0.003979951608926058
0  --->  3000
overlap:  {2181, 778, 907, 398, 1428, 2714, 681, 1963, 1710, 688, 1585, 1203, 2750, 2115, 325, 2760, 2505, 2635, 465, 1237, 1499, 993, 2021, 2663, 1767, 364, 1773, 880, 1136, 373, 2806}
@ shard: 1, @ slice: 3, @ epoch: 1, loss: 0.6

**performance using your proposed metric**

In [ ]:
print("unlearning time:",sisa_S5_R10.unlearning_time,"sec")
print("unlearning percent:",sisa_S5_R10.unlearning_percent*100,"%")

unlearning time: 733.5143201351166 sec
unlearning percent: 100.0 %


**Test Data:**

In [ ]:
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

Files already downloaded and verified



*  **weighted average aggregation:**

In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S5_R10.evaluate_with_weighted_aggregate(test_loader, sisa_S5_R10.aggregate_weighted_average, validation_loader)



all_preds= [0, 1, 0, 0, 5, 2, 8, 1, 3, 0, 0, 0, 9, 8, 6, 0, 8, 3, 5, 0, 5, 9, 7, 2, 6, 7, 0, 9, 6, 0, 6, 5, 9, 1, 4, 4, 5, 5, 1, 4, 9, 7, 1, 8, 1, 4, 6, 9, 0, 1, 2, 6, 1, 0, 8, 4, 9, 0, 9, 2, 6, 7, 6, 5, 1, 1, 7, 3, 2, 2, 0, 0, 0, 6, 5, 2, 1, 9, 6, 8, 4, 5, 3, 4, 2, 1, 6, 4, 4, 7, 7, 3, 6, 3, 5, 0, 0, 2, 1, 7, 0, 0, 3, 0, 3, 4, 7, 8, 8, 3, 4, 0, 8, 4, 4, 6, 2, 6, 6, 9, 4, 3, 6, 4, 3, 2, 9, 3, 4, 4, 5, 4, 9, 6, 6, 9, 7, 5, 0, 1, 2, 3, 4, 3, 1, 1, 5, 9, 8, 8, 6, 3, 3, 1, 0, 9, 5, 2, 6, 7, 7, 9, 1, 6, 2, 7, 5, 8, 0, 7, 8, 9, 9, 3, 5, 9, 4, 9, 8, 6, 8, 0, 6, 5, 0, 5, 8, 5, 5, 4, 9, 4, 3, 3, 7, 0, 9, 3, 8, 1, 0, 0, 6, 3, 5, 3, 6, 8, 1, 3, 4, 0, 2, 8, 1, 9, 2, 3, 8, 3, 0, 1, 4, 3, 1, 0, 6, 3, 6, 3, 6, 4, 0, 3, 1, 0, 5, 9, 9, 0, 0, 9, 5, 5, 1, 6, 2, 7, 0, 9, 4, 6, 3, 2, 2, 1, 6, 7, 3, 6, 7, 8, 0, 8, 4, 4, 9, 1, 6, 3, 2, 2, 7, 6, 2, 3, 4, 2, 4, 4, 7, 2, 2, 6, 6, 0, 2, 8, 9, 6, 2, 6, 1, 5, 3, 0, 2, 0, 2, 7, 7, 1, 7, 3, 6, 2, 2, 1, 8, 7, 8, 1, 8, 0, 1, 5, 1, 7, 0, 3, 5, 1, 8, 8, 7, 1, 2, 6, 8, 9

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9200, F1-score: 0.9199, Precision: 0.9211, Recall: 0.9200, AUROC: 0.9959



*  **average aggregation:**


In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R10.evaluate(test_loader, sisa_S5_R10.aggregate_simple_average)

all_preds= [0, 1, 0, 0, 5, 2, 8, 1, 3, 0, 0, 0, 9, 8, 6, 0, 8, 3, 5, 0, 5, 9, 7, 2, 6, 7, 0, 9, 6, 0, 6, 5, 9, 1, 4, 4, 5, 5, 1, 4, 9, 7, 1, 8, 1, 4, 6, 9, 0, 1, 2, 6, 1, 0, 8, 4, 9, 0, 9, 2, 6, 7, 6, 5, 1, 1, 7, 3, 2, 2, 0, 0, 0, 6, 5, 2, 1, 9, 6, 8, 4, 5, 3, 4, 2, 1, 6, 4, 4, 7, 7, 3, 6, 3, 5, 0, 0, 2, 1, 7, 0, 0, 3, 0, 3, 4, 7, 8, 8, 3, 4, 0, 8, 4, 4, 6, 2, 6, 6, 9, 4, 3, 6, 4, 3, 2, 9, 3, 4, 4, 5, 4, 9, 6, 6, 9, 7, 5, 0, 1, 2, 3, 4, 3, 1, 1, 5, 9, 8, 8, 6, 3, 3, 1, 0, 9, 5, 2, 6, 7, 7, 9, 1, 6, 2, 7, 5, 8, 0, 7, 8, 9, 9, 3, 5, 9, 4, 9, 8, 6, 8, 0, 6, 5, 0, 5, 8, 5, 5, 4, 9, 4, 3, 3, 7, 0, 9, 3, 8, 1, 0, 0, 6, 3, 5, 3, 6, 8, 1, 3, 4, 0, 2, 8, 1, 9, 2, 3, 8, 3, 0, 1, 4, 3, 1, 0, 6, 3, 6, 3, 6, 4, 0, 3, 1, 0, 5, 9, 9, 0, 0, 9, 5, 5, 1, 6, 2, 7, 0, 9, 4, 6, 3, 2, 2, 1, 6, 7, 3, 6, 7, 8, 0, 8, 4, 4, 9, 1, 6, 3, 2, 2, 7, 6, 2, 3, 4, 2, 4, 4, 7, 2, 2, 6, 6, 0, 2, 8, 9, 6, 2, 6, 1, 5, 3, 0, 2, 0, 2, 7, 7, 1, 7, 3, 6, 2, 2, 1, 8, 7, 8, 1, 8, 0, 1, 5, 1, 7, 0, 3, 5, 1, 8, 8, 7, 1, 2, 6, 8, 9

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9200, F1-score: 0.9199, Precision: 0.9211, Recall: 0.9200, AUROC: 0.9962


*  **majority vote aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R10.evaluate(test_loader, sisa_S5_R10.aggregate_majority_vote)

all_preds= [0, 1, 0, 0, 3, 2, 8, 1, 3, 0, 0, 0, 9, 8, 4, 0, 8, 3, 5, 0, 5, 9, 7, 2, 6, 7, 0, 9, 6, 0, 6, 5, 9, 1, 4, 4, 5, 5, 1, 4, 9, 7, 1, 8, 1, 4, 6, 9, 0, 1, 2, 6, 1, 0, 8, 4, 9, 0, 9, 2, 6, 5, 6, 5, 1, 1, 7, 3, 2, 2, 0, 0, 0, 6, 5, 2, 1, 9, 6, 8, 4, 5, 3, 4, 2, 1, 6, 4, 4, 7, 4, 3, 6, 3, 5, 0, 0, 2, 1, 7, 0, 0, 3, 0, 3, 4, 7, 8, 8, 3, 4, 0, 8, 4, 4, 6, 2, 6, 6, 9, 4, 7, 6, 4, 3, 2, 9, 3, 4, 4, 5, 4, 9, 6, 6, 9, 7, 5, 0, 1, 2, 3, 4, 3, 1, 1, 5, 9, 8, 8, 6, 3, 3, 1, 0, 9, 5, 2, 6, 7, 7, 9, 1, 6, 2, 7, 5, 8, 0, 7, 8, 9, 9, 3, 7, 9, 4, 9, 8, 6, 8, 0, 6, 5, 0, 5, 8, 5, 5, 4, 9, 4, 3, 3, 7, 0, 9, 3, 8, 1, 0, 0, 6, 3, 5, 3, 6, 8, 1, 3, 4, 0, 2, 8, 1, 9, 2, 3, 8, 3, 0, 1, 4, 3, 1, 0, 6, 3, 6, 3, 6, 4, 0, 3, 1, 0, 5, 9, 9, 0, 0, 9, 6, 5, 1, 6, 2, 7, 0, 9, 4, 6, 3, 2, 4, 1, 6, 7, 3, 6, 7, 8, 0, 8, 4, 4, 9, 1, 6, 3, 2, 2, 7, 6, 2, 3, 4, 2, 4, 4, 7, 2, 2, 6, 6, 0, 2, 8, 9, 6, 2, 6, 1, 5, 3, 0, 2, 0, 2, 7, 7, 1, 7, 3, 6, 2, 2, 1, 8, 7, 8, 1, 8, 0, 1, 5, 1, 7, 0, 3, 5, 1, 8, 8, 7, 1, 2, 6, 8, 9

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9130, F1-score: 0.9130, Precision: 0.9148, Recall: 0.9130, AUROC: 0.9849


## **S=5 R=20:**


**Initialize SISA**


In [ ]:
S = 5
R = 20
epochs = 5

sisa_S5_R20 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:

# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1oksdaxjVrjOjAtNcbxRvNhxvgWEeBLPe'

# Download and extract models
sisa_S5_R20.download_and_extract_models_from_drive(file_id)


Downloaded models_S5_R20.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


**load model from zip file:**

In [ ]:
sisa_S5_R20.extract_and_load_model('/content/models_S5_R20.zip')

**Unlearning:**

In [ ]:
sisa_S5_R20.unlearn(cifar10_train, 500)

0  --->  500
overlap:  {346, 267, 52, 455}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1, @ slice: 1, @ epoch: 1, loss: 2.139878511428833
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 1.7507295608520508
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 0.9546127319335938
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 0.55352383852005
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 0.2444646805524826
0  --->  1000
overlap:  {803, 838, 455, 967, 267, 908, 782, 943, 52, 346}
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 0.6097409129142761
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 0.4776751697063446
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.2176256775856018
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 0.08576484769582748
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 0.09281298518180847
0  --->  1500
overlap:  {803, 838, 455, 1192, 967, 1098, 267, 908, 1289, 782, 943, 52, 1143, 346}
@ shard: 1, @ slice: 3, @ epoch: 1, loss: 0.49195900559425354
@ shard: 1, @ slice: 3, @ epoch: 2, loss: 0.2946363091468811
@ shard: 1, @ slice: 3, @ epoch: 3, loss: 0.06274159997701645
@ shard: 1, @ slice: 3, @ epoch: 4

**performance using your proposed metric**

In [ ]:
print("unlearning time:",sisa_S5_R20.unlearning_time,"sec")
print("unlearning percent:",sisa_S5_R20.unlearning_percent*100,"%")

unlearning time: 785.7263462543488 sec
unlearning percent: 100.0 %


**Test Data:**

In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Get a sample of 100 test examples
sample_indices = torch.randperm(len(cifar10_test))[:1000]
test_data = torch.utils.data.Subset(cifar10_test, sample_indices)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

Files already downloaded and verified



*  **weighted average aggregation:**


In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S5_R20.evaluate_with_weighted_aggregate(test_loader, sisa_S5_R20.aggregate_weighted_average, validation_loader)



all_preds= [1, 6, 9, 1, 5, 9, 4, 6, 5, 7, 5, 6, 8, 5, 4, 5, 2, 3, 4, 4, 3, 8, 1, 6, 5, 0, 7, 6, 1, 5, 6, 3, 7, 3, 3, 4, 6, 6, 1, 0, 0, 1, 5, 0, 8, 2, 2, 2, 8, 6, 7, 0, 3, 5, 4, 4, 4, 4, 4, 5, 7, 4, 1, 1, 0, 9, 1, 5, 0, 4, 9, 9, 5, 3, 4, 1, 0, 9, 0, 7, 9, 7, 1, 8, 0, 3, 8, 5, 7, 4, 9, 4, 8, 2, 5, 2, 0, 9, 4, 3, 2, 7, 0, 1, 1, 4, 9, 5, 3, 7, 0, 8, 0, 5, 0, 9, 9, 7, 8, 7, 2, 5, 6, 5, 4, 7, 7, 5, 9, 5, 7, 3, 9, 4, 9, 2, 0, 0, 3, 6, 7, 4, 6, 4, 2, 0, 4, 8, 8, 6, 2, 6, 0, 9, 9, 5, 0, 1, 1, 5, 7, 2, 6, 9, 3, 7, 1, 9, 1, 4, 3, 6, 6, 2, 5, 9, 6, 2, 1, 8, 2, 4, 8, 6, 2, 4, 9, 5, 0, 9, 6, 4, 6, 1, 1, 0, 7, 9, 0, 3, 5, 1, 6, 9, 1, 8, 7, 9, 4, 4, 3, 1, 1, 1, 8, 4, 7, 2, 4, 2, 6, 4, 1, 5, 5, 1, 9, 9, 1, 6, 4, 2, 1, 2, 1, 0, 2, 5, 3, 9, 2, 9, 1, 0, 8, 6, 0, 9, 9, 7, 2, 2, 4, 9, 0, 1, 9, 2, 2, 9, 1, 3, 9, 3, 5, 7, 7, 1, 1, 9, 9, 2, 2, 3, 4, 6, 8, 9, 8, 1, 0, 7, 4, 7, 2, 9, 6, 7, 3, 6, 3, 4, 6, 7, 2, 2, 8, 8, 2, 1, 7, 6, 8, 6, 3, 1, 1, 0, 9, 3, 4, 4, 7, 2, 7, 1, 9, 7, 2, 8, 6, 9, 6, 2, 1, 1, 4, 9, 3, 7

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9050, F1-score: 0.9047, Precision: 0.9055, Recall: 0.9050, AUROC: 0.9945


*  **average aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R20.evaluate(test_loader, sisa_S5_R20.aggregate_simple_average)

all_preds= [1, 6, 9, 1, 5, 9, 4, 6, 5, 7, 5, 6, 8, 5, 4, 5, 2, 3, 4, 4, 3, 8, 1, 6, 5, 0, 7, 6, 1, 5, 6, 3, 7, 3, 3, 4, 6, 6, 1, 0, 0, 1, 5, 0, 8, 2, 2, 2, 8, 6, 7, 0, 3, 5, 4, 4, 4, 4, 4, 5, 7, 4, 1, 1, 0, 9, 1, 5, 0, 4, 9, 9, 5, 3, 4, 1, 0, 9, 0, 7, 9, 7, 1, 8, 0, 3, 8, 5, 7, 4, 9, 4, 8, 2, 5, 2, 0, 9, 4, 3, 2, 7, 0, 1, 1, 4, 9, 5, 3, 7, 0, 8, 0, 5, 0, 9, 9, 7, 8, 7, 2, 5, 6, 5, 4, 7, 7, 5, 9, 5, 7, 3, 9, 4, 9, 2, 0, 0, 3, 6, 7, 4, 6, 4, 2, 0, 4, 8, 8, 6, 2, 6, 0, 9, 9, 5, 0, 1, 1, 5, 7, 2, 6, 9, 3, 7, 1, 9, 1, 4, 3, 6, 6, 2, 5, 9, 6, 2, 1, 8, 2, 4, 8, 6, 2, 4, 9, 5, 0, 9, 6, 4, 6, 1, 1, 0, 7, 9, 0, 3, 5, 1, 6, 9, 1, 8, 7, 9, 4, 4, 3, 1, 1, 1, 8, 4, 7, 2, 4, 2, 6, 4, 1, 5, 5, 1, 9, 9, 1, 6, 4, 2, 1, 2, 1, 0, 2, 5, 3, 9, 2, 9, 1, 0, 8, 6, 0, 9, 9, 7, 2, 2, 4, 9, 0, 1, 9, 2, 2, 9, 1, 3, 9, 3, 5, 7, 7, 1, 1, 9, 9, 2, 2, 3, 4, 6, 8, 9, 8, 1, 0, 7, 4, 7, 2, 9, 6, 7, 3, 6, 3, 4, 6, 7, 2, 2, 8, 8, 2, 1, 7, 6, 8, 6, 3, 1, 1, 0, 9, 3, 4, 4, 7, 2, 7, 1, 9, 7, 2, 8, 6, 9, 6, 2, 1, 1, 4, 9, 3, 7

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9050, F1-score: 0.9047, Precision: 0.9055, Recall: 0.9050, AUROC: 0.9948



*  **majority vote aggregation:**


In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S5_R20.evaluate(test_loader, sisa_S5_R20.aggregate_majority_vote)

all_preds= [1, 6, 9, 1, 5, 9, 4, 6, 5, 7, 5, 6, 8, 5, 4, 5, 2, 3, 4, 4, 3, 8, 1, 6, 5, 0, 7, 6, 1, 5, 6, 3, 7, 3, 3, 4, 6, 6, 0, 0, 0, 1, 5, 0, 8, 2, 2, 2, 8, 6, 7, 0, 3, 5, 4, 4, 4, 0, 4, 3, 7, 4, 1, 1, 0, 9, 1, 5, 0, 4, 9, 9, 5, 3, 4, 1, 0, 9, 0, 7, 9, 7, 1, 8, 0, 3, 8, 5, 7, 4, 9, 4, 8, 2, 5, 2, 0, 9, 4, 3, 2, 7, 0, 1, 1, 4, 9, 5, 3, 7, 0, 8, 0, 5, 0, 9, 9, 7, 8, 7, 2, 5, 6, 5, 4, 7, 7, 3, 9, 5, 7, 6, 9, 4, 9, 2, 0, 0, 3, 6, 7, 4, 4, 4, 2, 0, 4, 8, 8, 6, 2, 6, 0, 9, 9, 5, 0, 1, 1, 5, 7, 2, 6, 9, 3, 7, 1, 9, 1, 4, 5, 6, 6, 2, 5, 9, 6, 2, 1, 8, 2, 4, 8, 3, 2, 4, 9, 5, 0, 9, 6, 4, 6, 1, 1, 0, 7, 9, 0, 3, 5, 1, 6, 9, 1, 8, 7, 0, 4, 4, 3, 1, 1, 1, 8, 3, 7, 2, 7, 2, 6, 4, 1, 5, 5, 1, 9, 9, 1, 6, 4, 2, 1, 2, 1, 0, 2, 5, 3, 9, 2, 9, 1, 0, 8, 6, 0, 9, 9, 7, 4, 2, 4, 9, 0, 1, 9, 0, 2, 9, 1, 3, 9, 3, 2, 7, 7, 1, 1, 9, 9, 2, 2, 3, 4, 6, 8, 9, 8, 1, 0, 7, 4, 7, 2, 9, 6, 7, 3, 5, 3, 4, 6, 7, 2, 2, 8, 8, 2, 1, 7, 6, 8, 6, 3, 1, 0, 0, 9, 3, 4, 4, 7, 2, 7, 1, 9, 7, 2, 8, 6, 9, 6, 2, 1, 1, 4, 9, 3, 7

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.8930, F1-score: 0.8929, Precision: 0.8938, Recall: 0.8930, AUROC: 0.9816


## **S=10 R=5:**

**Initialize SISA**

In [ ]:
S = 10
R = 5
epochs = 5

sisa_S10_R5 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:

# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1muwLWjXoCIQiVcwq5F3aqZEo6kuriMei'

# Download and extract models
sisa_S10_R5.download_and_extract_models_from_drive(file_id)


Downloaded models_S10_R5.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5
loading shard: 6
loading shard: 7
loading shard: 8
loading shard: 9
loading shard: 10



**load model from zip file:**


In [ ]:
sisa_S10_R5.extract_and_load_model('/content/models_S10_R5.zip')

**Unlearning:**

In [ ]:
sisa_S10_R5.unlearn(cifar10_train, 500)

0  --->  1000
overlap:  {9, 170, 398, 882, 371}
@ shard: 1, @ slice: 1, @ epoch: 1, loss: 1.9688293933868408
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 1.1122328042984009
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 0.5203711986541748
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 0.178401380777359
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 0.05602959170937538
0  --->  2000
overlap:  {1225, 170, 9, 398, 1646, 1361, 882, 371, 1555, 1107, 1556, 1304, 1404}
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 0.4991602897644043
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 0.32820379734039307
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.09604246914386749
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 0.04249419644474983
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 0.00805174931883812
0  --->  3000
overlap:  {9, 398, 1555, 1556, 1304, 2713, 2340, 170, 2090, 2223, 2633, 1225, 1361, 2131, 1107, 2527, 1646, 882, 371, 2041, 1404, 2174}
@ shard: 1, @ slice: 3, @ epoch: 1, loss: 0.5903121829032898
@ shard: 1, @ slice: 3, @ epoch

**performance using your proposed metric**

In [ ]:
print("unlearning time:",sisa_S10_R5.unlearning_time,"sec")
print("unlearning percent:",sisa_S10_R5.unlearning_percent*100,"%")

unlearning time: 746.9020791053772 sec
unlearning percent: 100.0 %


**Test Data:**

In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:
# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S10_R5.evaluate_with_weighted_aggregate(test_loader, sisa_S10_R5.aggregate_weighted_average, validation_loader)

all_preds= [5, 1, 3, 7, 1, 6, 2, 2, 5, 9, 0, 1, 9, 0, 0, 6, 5, 2, 1, 3, 1, 7, 1, 3, 4, 1, 3, 2, 6, 8, 7, 3, 9, 3, 9, 4, 8, 1, 7, 1, 6, 2, 0, 2, 3, 6, 5, 0, 1, 5, 1, 3, 8, 5, 5, 8, 7, 9, 8, 0, 4, 4, 6, 1, 5, 3, 4, 7, 3, 4, 9, 5, 2, 2, 9, 5, 3, 4, 0, 1, 1, 7, 7, 0, 6, 7, 5, 5, 3, 6, 4, 2, 0, 3, 0, 5, 8, 2, 8, 7, 3, 5, 6, 3, 0, 1, 6, 9, 4, 1, 4, 2, 3, 1, 9, 5, 5, 5, 0, 2, 8, 9, 5, 1, 9, 1, 0, 7, 0, 6, 7, 7, 0, 4, 0, 5, 1, 6, 3, 7, 6, 5, 8, 9, 2, 8, 6, 3, 3, 9, 2, 2, 0, 1, 5, 0, 4, 4, 7, 5, 2, 4, 3, 7, 6, 4, 9, 9, 1, 5, 0, 3, 6, 2, 4, 9, 9, 2, 8, 0, 0, 4, 3, 3, 1, 6, 6, 4, 7, 4, 3, 6, 8, 7, 1, 3, 9, 6, 3, 1, 7, 0, 9, 9, 8, 6, 5, 0, 5, 0, 6, 0, 0, 7, 9, 2, 5, 0, 8, 4, 0, 5, 7, 7, 0, 7, 8, 5, 4, 6, 4, 1, 0, 0, 3, 9, 4, 2, 0, 9, 9, 7, 1, 4, 2, 9, 3, 9, 3, 4, 9, 4, 9, 0, 0, 6, 3, 7, 8, 0, 1, 5, 2, 2, 2, 2, 9, 5, 1, 6, 6, 8, 1, 9, 3, 3, 2, 3, 7, 6, 5, 5, 0, 4, 3, 9, 4, 5, 6, 3, 0, 0, 6, 4, 9, 3, 7, 5, 1, 0, 6, 6, 6, 0, 9, 7, 6, 2, 3, 2, 0, 3, 4, 7, 5, 6, 7, 8, 6, 9, 0, 4, 9, 8, 4, 6, 4, 8, 8, 7

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9300, F1-score: 0.9299, Precision: 0.9307, Recall: 0.9300, AUROC: 0.9967


*  **average aggregation:**

In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S10_R5.evaluate(test_loader, sisa_S10_R5.aggregate_simple_average)


all_preds= [5, 1, 3, 7, 1, 6, 2, 2, 5, 9, 0, 1, 9, 0, 0, 6, 5, 2, 1, 3, 1, 7, 1, 3, 4, 1, 3, 2, 6, 8, 7, 3, 9, 3, 9, 4, 8, 1, 7, 1, 6, 2, 0, 2, 3, 6, 5, 0, 1, 5, 1, 3, 8, 5, 5, 8, 7, 9, 8, 0, 4, 4, 6, 1, 5, 3, 4, 7, 3, 4, 9, 5, 2, 2, 9, 5, 3, 4, 0, 1, 1, 7, 7, 0, 6, 7, 5, 5, 3, 6, 4, 2, 0, 3, 0, 5, 8, 2, 8, 7, 3, 5, 6, 3, 0, 1, 6, 9, 4, 1, 4, 2, 3, 1, 9, 5, 5, 5, 0, 2, 8, 9, 5, 1, 9, 1, 0, 7, 0, 6, 7, 7, 0, 4, 0, 5, 1, 6, 3, 7, 6, 5, 8, 9, 2, 8, 6, 3, 3, 9, 2, 2, 0, 1, 5, 0, 4, 4, 7, 5, 2, 4, 3, 7, 6, 4, 9, 9, 1, 5, 0, 3, 6, 2, 4, 9, 9, 2, 8, 0, 0, 4, 3, 3, 1, 6, 6, 4, 7, 4, 3, 6, 8, 7, 1, 3, 9, 6, 3, 1, 7, 0, 9, 9, 8, 6, 5, 0, 5, 0, 6, 0, 0, 7, 9, 2, 5, 0, 8, 4, 0, 5, 7, 7, 0, 7, 8, 5, 4, 6, 4, 1, 0, 0, 3, 9, 4, 2, 0, 9, 9, 7, 1, 4, 2, 9, 3, 9, 3, 4, 9, 4, 9, 0, 0, 6, 3, 7, 8, 0, 1, 5, 2, 2, 2, 2, 9, 5, 1, 6, 6, 8, 1, 9, 3, 3, 2, 3, 7, 6, 5, 5, 0, 4, 3, 9, 4, 5, 6, 3, 0, 0, 6, 4, 9, 3, 7, 5, 1, 0, 6, 6, 6, 0, 9, 7, 6, 2, 3, 2, 0, 3, 4, 7, 5, 6, 7, 8, 6, 9, 0, 4, 9, 8, 4, 6, 4, 8, 8, 7

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9300, F1-score: 0.9299, Precision: 0.9307, Recall: 0.9300, AUROC: 0.9975




*  **majority vote aggregation:**


In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S10_R5.evaluate(test_loader, sisa_S10_R5.aggregate_majority_vote)

all_preds= [5, 1, 3, 7, 1, 6, 2, 2, 5, 9, 0, 1, 9, 0, 0, 6, 5, 2, 1, 3, 1, 7, 1, 3, 4, 1, 3, 2, 6, 8, 7, 3, 9, 5, 9, 4, 8, 1, 7, 1, 6, 2, 0, 2, 3, 6, 5, 0, 1, 5, 1, 3, 8, 5, 5, 8, 7, 9, 8, 0, 4, 4, 6, 1, 5, 3, 4, 7, 3, 4, 9, 5, 2, 2, 9, 5, 3, 4, 0, 1, 1, 7, 7, 0, 6, 7, 5, 5, 3, 6, 4, 2, 0, 3, 0, 5, 8, 2, 8, 7, 3, 5, 6, 3, 0, 1, 6, 9, 4, 1, 4, 2, 3, 1, 9, 5, 5, 5, 0, 2, 8, 9, 5, 1, 9, 1, 0, 7, 0, 6, 7, 7, 0, 4, 0, 5, 1, 6, 3, 7, 6, 5, 8, 9, 2, 8, 6, 3, 3, 9, 2, 2, 0, 1, 5, 0, 4, 4, 7, 5, 2, 4, 3, 7, 6, 4, 9, 9, 1, 5, 0, 3, 4, 2, 4, 9, 9, 2, 8, 0, 0, 4, 3, 3, 1, 6, 6, 4, 7, 4, 3, 6, 8, 7, 1, 3, 9, 6, 3, 1, 7, 0, 9, 9, 8, 6, 5, 0, 5, 0, 6, 0, 0, 7, 9, 2, 7, 0, 8, 4, 0, 5, 7, 7, 0, 7, 8, 5, 4, 6, 4, 1, 0, 0, 3, 9, 4, 2, 0, 9, 9, 7, 1, 4, 2, 9, 3, 9, 3, 4, 9, 4, 9, 0, 0, 6, 7, 7, 8, 0, 1, 5, 2, 2, 4, 2, 9, 5, 1, 6, 6, 8, 1, 9, 3, 3, 2, 3, 7, 6, 5, 5, 0, 4, 0, 9, 4, 5, 6, 3, 0, 0, 6, 4, 9, 3, 7, 5, 1, 0, 6, 6, 6, 0, 9, 7, 6, 2, 3, 2, 0, 3, 4, 7, 5, 6, 7, 8, 6, 9, 0, 4, 9, 8, 4, 6, 4, 8, 8, 7

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9230, F1-score: 0.9228, Precision: 0.9237, Recall: 0.9230, AUROC: 0.9948


## **S=10 R=10:**



**Initialize SISA**

In [ ]:
S = 10
R = 10
epochs = 5

sisa_S10_R10 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:

# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1dCWYPHZbQHD81XLNB82aaNoQxoCVGNBG'

# Download and extract models
sisa_S10_R10.download_and_extract_models_from_drive(file_id)


Downloaded models_S10_R10.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5
loading shard: 6
loading shard: 7
loading shard: 8
loading shard: 9
loading shard: 10



**load model from zip file:**


In [ ]:
sisa_S10_R10.extract_and_load_model('/content/models_S10_R10.zip')


**Unlearning:**


In [ ]:
sisa_S10_R10.unlearn(cifar10_train, 500)

0  --->  500
overlap:  {330, 494, 47, 371, 473}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1, @ slice: 1, @ epoch: 1, loss: 2.2179341316223145
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 1.777955412864685
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 1.070854902267456
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 0.5483556985855103
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 0.22028495371341705
0  --->  1000
overlap:  {578, 867, 330, 494, 47, 560, 371, 473, 927}
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 0.8780544996261597
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 0.5831475853919983
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.1712842434644699
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 0.06723585724830627
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 0.044741515070199966
0  --->  1500
overlap:  {578, 867, 330, 494, 1039, 560, 47, 1105, 371, 1045, 1304, 473, 927}
@ shard: 1, @ slice: 3, @ epoch: 1, loss: 0.6952027082443237
@ shard: 1, @ slice: 3, @ epoch: 2, loss: 0.26540589332580566
@ shard: 1, @ slice: 3, @ epoch: 3, loss: 0.07384208589792252
@ shard: 1, @ slice: 3, @ epoch: 4, loss:


**performance using your proposed metric**


In [ ]:

print("unlearning time:",sisa_S10_R10.unlearning_time,"sec")
print("unlearning percent:",sisa_S10_R10.unlearning_percent*100,"%")


unlearning time: 785.5714693069458 sec
unlearning percent: 100.0 %



**Test Data:**


In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)


Files already downloaded and verified



*  **weighted average aggregation:**


In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S10_R10.evaluate_with_weighted_aggregate(test_loader, sisa_S10_R10.aggregate_weighted_average, validation_loader)


all_preds= [0, 2, 4, 9, 0, 3, 3, 3, 2, 8, 5, 7, 9, 6, 5, 6, 4, 6, 8, 9, 6, 9, 1, 7, 4, 0, 5, 1, 5, 0, 6, 4, 7, 3, 2, 0, 8, 6, 6, 7, 3, 5, 1, 7, 2, 8, 5, 3, 4, 3, 8, 3, 2, 5, 5, 4, 3, 7, 4, 0, 7, 8, 2, 3, 3, 1, 7, 3, 9, 5, 8, 0, 7, 5, 6, 4, 5, 2, 4, 6, 2, 1, 3, 4, 5, 6, 0, 0, 1, 1, 0, 6, 0, 2, 6, 1, 6, 2, 8, 1, 2, 5, 7, 1, 9, 9, 9, 5, 7, 7, 2, 3, 6, 4, 6, 7, 7, 1, 1, 1, 5, 6, 7, 2, 0, 8, 8, 1, 5, 4, 5, 1, 1, 1, 2, 9, 3, 1, 8, 6, 6, 9, 3, 8, 1, 6, 5, 1, 3, 0, 9, 7, 2, 5, 7, 2, 1, 5, 0, 5, 4, 1, 1, 2, 8, 5, 2, 8, 8, 3, 0, 5, 9, 6, 7, 3, 2, 7, 2, 6, 3, 6, 8, 4, 3, 1, 8, 4, 9, 5, 7, 6, 5, 4, 4, 1, 7, 8, 1, 6, 6, 6, 6, 1, 5, 5, 0, 6, 0, 9, 9, 6, 8, 4, 4, 6, 6, 4, 8, 6, 1, 2, 6, 0, 3, 2, 2, 3, 4, 3, 0, 3, 1, 3, 5, 4, 4, 2, 7, 0, 7, 7, 3, 9, 2, 4, 0, 5, 7, 8, 1, 9, 8, 9, 2, 0, 4, 6, 8, 9, 5, 2, 7, 5, 4, 2, 6, 2, 9, 9, 0, 5, 3, 7, 9, 5, 1, 7, 7, 1, 3, 6, 8, 5, 9, 1, 8, 4, 8, 1, 7, 7, 5, 9, 9, 2, 0, 5, 0, 0, 2, 0, 8, 8, 4, 6, 6, 3, 6, 2, 6, 8, 4, 6, 8, 0, 6, 3, 1, 7, 1, 3, 2, 1, 4, 4, 1, 9, 9, 4

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9090, F1-score: 0.9093, Precision: 0.9110, Recall: 0.9090, AUROC: 0.9954


*  **average aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S10_R10.evaluate(test_loader, sisa_S10_R10.aggregate_simple_average)

all_preds= [0, 2, 4, 9, 0, 3, 3, 3, 2, 8, 5, 7, 9, 6, 5, 6, 4, 6, 8, 9, 6, 9, 1, 7, 4, 0, 5, 1, 5, 0, 6, 4, 7, 3, 2, 0, 8, 6, 6, 7, 3, 5, 1, 7, 2, 8, 5, 3, 4, 3, 8, 3, 2, 5, 5, 4, 3, 7, 4, 0, 7, 8, 2, 3, 3, 1, 7, 3, 9, 5, 8, 0, 7, 5, 6, 4, 5, 2, 4, 6, 2, 1, 3, 4, 5, 6, 0, 0, 1, 1, 0, 6, 0, 2, 6, 1, 6, 0, 8, 1, 2, 5, 7, 1, 9, 9, 9, 5, 7, 7, 2, 3, 6, 4, 6, 7, 7, 1, 1, 1, 5, 6, 7, 2, 0, 8, 8, 1, 5, 4, 5, 1, 1, 1, 2, 9, 3, 1, 8, 6, 6, 9, 3, 8, 1, 6, 5, 1, 3, 0, 9, 7, 2, 5, 7, 2, 1, 5, 0, 5, 4, 1, 1, 2, 8, 5, 2, 8, 8, 3, 0, 5, 9, 6, 7, 3, 2, 7, 2, 6, 3, 6, 8, 4, 3, 1, 8, 4, 9, 5, 7, 6, 5, 4, 4, 1, 7, 8, 1, 6, 6, 6, 6, 1, 5, 5, 0, 6, 0, 9, 9, 6, 8, 4, 4, 6, 6, 4, 8, 6, 1, 2, 6, 0, 3, 2, 2, 3, 4, 3, 0, 3, 1, 3, 5, 4, 4, 2, 7, 0, 7, 7, 3, 9, 2, 4, 0, 5, 7, 8, 1, 9, 8, 9, 2, 0, 4, 6, 8, 9, 5, 2, 7, 5, 4, 2, 6, 2, 9, 9, 0, 5, 3, 7, 9, 5, 1, 7, 7, 1, 3, 6, 8, 5, 9, 1, 8, 4, 8, 1, 7, 7, 5, 9, 9, 2, 0, 5, 0, 0, 2, 0, 8, 8, 4, 6, 6, 3, 6, 2, 6, 8, 4, 6, 8, 0, 6, 3, 1, 7, 1, 3, 2, 1, 4, 4, 1, 9, 9, 4

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9100, F1-score: 0.9102, Precision: 0.9120, Recall: 0.9100, AUROC: 0.9962


*  **majority vote aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S10_R10.evaluate(test_loader, sisa_S10_R10.aggregate_majority_vote)

all_preds= [0, 2, 4, 9, 0, 3, 3, 3, 2, 8, 5, 7, 9, 6, 5, 6, 4, 6, 8, 9, 6, 9, 1, 7, 4, 0, 3, 1, 5, 0, 6, 4, 7, 3, 2, 0, 8, 6, 6, 7, 3, 4, 1, 7, 2, 8, 5, 3, 4, 3, 8, 3, 2, 5, 5, 4, 3, 7, 4, 8, 7, 0, 2, 3, 3, 1, 7, 3, 9, 5, 8, 0, 3, 5, 6, 4, 5, 2, 3, 6, 2, 1, 3, 4, 5, 6, 0, 0, 1, 1, 0, 6, 0, 2, 6, 1, 6, 0, 8, 1, 2, 5, 7, 1, 9, 9, 9, 5, 7, 7, 2, 3, 6, 4, 6, 7, 7, 1, 1, 1, 5, 6, 7, 2, 0, 8, 8, 1, 5, 4, 5, 1, 1, 1, 2, 9, 3, 1, 8, 6, 6, 9, 3, 8, 1, 6, 5, 1, 3, 0, 9, 7, 2, 5, 7, 2, 1, 5, 0, 5, 4, 1, 1, 2, 8, 5, 2, 8, 8, 3, 0, 5, 9, 6, 7, 3, 2, 7, 2, 6, 3, 6, 8, 4, 3, 1, 8, 4, 9, 5, 7, 6, 5, 4, 4, 1, 7, 8, 1, 6, 6, 6, 6, 1, 5, 5, 0, 6, 0, 9, 9, 6, 8, 4, 4, 6, 6, 4, 8, 6, 1, 2, 6, 0, 3, 2, 2, 3, 4, 3, 0, 3, 1, 3, 5, 4, 2, 2, 7, 0, 7, 7, 3, 9, 2, 4, 0, 5, 7, 8, 1, 9, 8, 9, 2, 0, 4, 6, 8, 9, 5, 2, 7, 5, 3, 2, 6, 2, 9, 9, 0, 5, 3, 7, 9, 5, 1, 7, 7, 1, 3, 6, 8, 5, 9, 1, 8, 4, 8, 1, 7, 7, 5, 9, 9, 2, 0, 5, 0, 0, 2, 0, 8, 8, 4, 6, 6, 3, 6, 2, 6, 8, 2, 6, 8, 0, 2, 3, 1, 7, 1, 3, 2, 1, 4, 0, 1, 9, 9, 4

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9110, F1-score: 0.9112, Precision: 0.9133, Recall: 0.9110, AUROC: 0.9911


## **S=10 R=20:**


**Initialize SISA**

In [ ]:
S = 10
R = 20
epochs = 5

sisa_S10_R20 = SISA(S, R, epochs)


**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1x0MfTN5ztDCumz2nBiUgnRvG444v2TUp'

# Download and extract models
sisa_S10_R20.download_and_extract_models_from_drive(file_id)

Downloaded models_S10_R20.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5
loading shard: 6
loading shard: 7
loading shard: 8
loading shard: 9
loading shard: 10



**load model from zip file:**

In [ ]:

sisa_S10_R20.extract_and_load_model('/content/models_S10_R20.zip')



**Unlearning:**


In [ ]:

sisa_S10_R20.unlearn(cifar10_train, 500)

0  --->  250
overlap:  {162, 60}
@ shard: 1, @ slice: 1, @ epoch: 1, loss: 2.279772996902466
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 2.221956253051758
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 1.8788915872573853
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 1.358312726020813
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 1.1144074201583862
0  --->  500
overlap:  {162, 60, 279}
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 1.4261623620986938
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 1.1373916864395142
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.617418110370636
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 0.3843064308166504
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 0.1719033420085907
0  --->  750
overlap:  {641, 162, 679, 746, 279, 60}
@ shard: 1, @ slice: 3, @ epoch: 1, loss: 0.8583915829658508
@ shard: 1, @ slice: 3, @ epoch: 2, loss: 0.4208233952522278
@ shard: 1, @ slice: 3, @ epoch: 3, loss: 0.26012739539146423
@ shard: 1, @ slice: 3, @ epoch: 4, loss: 0.1282566338777542
@ shard: 1, @ slice: 3, @


**performance using your proposed metric**


In [ ]:

print("unlearning time:",sisa_S10_R20.unlearning_time,"sec")
print("unlearning percent:",sisa_S10_R20.unlearning_percent*100,"%")

unlearning time: 863.4429023265839 sec
unlearning percent: 98.5 %


**Test Data:**


In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)


Files already downloaded and verified


*  **weighted average aggregation:**

In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S10_R20.evaluate_with_weighted_aggregate(test_loader, sisa_S10_R20.aggregate_weighted_average, validation_loader)


all_preds= [3, 8, 7, 5, 3, 1, 8, 4, 1, 4, 4, 4, 8, 7, 8, 7, 3, 7, 3, 7, 9, 2, 9, 9, 1, 2, 0, 3, 0, 2, 8, 2, 6, 1, 2, 3, 6, 2, 5, 3, 8, 9, 9, 0, 9, 9, 6, 0, 9, 9, 8, 1, 4, 1, 5, 5, 8, 1, 4, 7, 5, 9, 0, 8, 9, 5, 5, 9, 9, 3, 8, 0, 5, 8, 3, 1, 6, 3, 5, 8, 8, 4, 7, 9, 7, 7, 4, 1, 9, 5, 5, 0, 4, 3, 8, 7, 7, 8, 5, 0, 4, 8, 7, 4, 4, 5, 2, 5, 0, 0, 5, 3, 6, 5, 4, 3, 0, 9, 1, 4, 9, 0, 2, 1, 7, 4, 4, 2, 0, 7, 4, 2, 7, 1, 3, 5, 7, 1, 7, 2, 5, 2, 1, 4, 1, 6, 0, 1, 0, 7, 4, 5, 5, 3, 6, 1, 9, 3, 7, 1, 6, 2, 3, 9, 9, 7, 1, 0, 5, 8, 0, 6, 9, 8, 8, 6, 9, 0, 8, 3, 2, 7, 3, 1, 9, 8, 3, 3, 7, 3, 8, 8, 2, 9, 7, 4, 5, 3, 5, 5, 0, 9, 7, 0, 6, 9, 5, 2, 2, 2, 9, 1, 4, 3, 6, 9, 6, 5, 2, 8, 6, 3, 1, 0, 5, 5, 6, 7, 1, 9, 7, 6, 1, 7, 8, 7, 4, 9, 1, 3, 1, 2, 6, 7, 2, 7, 3, 8, 0, 9, 0, 4, 3, 5, 5, 1, 8, 4, 6, 4, 5, 2, 5, 2, 3, 6, 7, 7, 7, 8, 6, 3, 0, 4, 3, 6, 7, 5, 6, 1, 3, 4, 9, 3, 6, 8, 9, 9, 3, 1, 4, 7, 2, 7, 4, 7, 7, 0, 7, 7, 2, 4, 7, 6, 1, 3, 5, 0, 9, 0, 1, 0, 3, 4, 3, 4, 4, 2, 1, 4, 6, 4, 7, 0, 4, 7, 1, 1, 1, 9

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9180, F1-score: 0.9180, Precision: 0.9186, Recall: 0.9180, AUROC: 0.9937


*  **average aggregation:**

In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S10_R20.evaluate(test_loader, sisa_S10_R20.aggregate_simple_average)

all_preds= [3, 8, 7, 5, 3, 1, 8, 4, 1, 4, 4, 4, 8, 7, 8, 7, 3, 7, 3, 7, 9, 2, 9, 9, 1, 2, 0, 3, 0, 2, 8, 2, 6, 1, 2, 3, 6, 2, 5, 3, 8, 9, 9, 0, 9, 9, 6, 0, 9, 9, 8, 1, 4, 1, 5, 5, 8, 1, 4, 7, 5, 9, 0, 8, 9, 5, 5, 9, 9, 3, 8, 0, 5, 8, 3, 1, 6, 3, 5, 8, 8, 4, 7, 9, 7, 7, 4, 1, 9, 5, 5, 0, 4, 3, 8, 7, 7, 8, 5, 0, 4, 8, 7, 4, 4, 5, 2, 5, 0, 0, 5, 3, 6, 5, 4, 3, 0, 9, 1, 4, 9, 0, 2, 1, 7, 4, 4, 2, 0, 7, 4, 2, 7, 1, 3, 5, 7, 1, 7, 2, 5, 2, 1, 4, 1, 6, 0, 1, 0, 7, 4, 5, 5, 3, 6, 1, 9, 3, 7, 1, 6, 2, 3, 9, 9, 7, 1, 0, 5, 8, 0, 6, 9, 8, 8, 6, 9, 0, 8, 3, 2, 7, 3, 1, 9, 8, 3, 3, 7, 3, 8, 8, 2, 9, 7, 4, 5, 3, 5, 5, 0, 9, 7, 0, 6, 9, 5, 2, 2, 2, 9, 1, 4, 3, 6, 9, 6, 5, 2, 8, 6, 3, 1, 0, 5, 5, 6, 7, 1, 9, 7, 6, 1, 7, 8, 7, 4, 9, 1, 3, 1, 2, 6, 7, 2, 7, 3, 8, 0, 9, 0, 4, 3, 5, 5, 1, 8, 4, 6, 4, 5, 2, 5, 2, 3, 6, 7, 7, 7, 8, 6, 3, 0, 4, 3, 6, 7, 5, 6, 1, 3, 4, 9, 3, 6, 8, 9, 9, 3, 1, 4, 5, 2, 7, 4, 7, 7, 0, 7, 7, 2, 4, 7, 6, 1, 3, 5, 0, 9, 0, 1, 0, 3, 4, 3, 4, 4, 2, 1, 4, 6, 4, 7, 0, 4, 7, 1, 1, 1, 9

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9180, F1-score: 0.9180, Precision: 0.9187, Recall: 0.9180, AUROC: 0.9946




*  **majority vote aggregation:**


In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S10_R20.evaluate(test_loader, sisa_S10_R20.aggregate_majority_vote)

all_preds= [3, 8, 7, 5, 3, 1, 8, 4, 1, 4, 4, 4, 8, 7, 8, 7, 3, 7, 3, 7, 9, 2, 9, 9, 1, 2, 0, 3, 0, 2, 8, 2, 6, 1, 2, 3, 6, 2, 5, 3, 8, 9, 9, 8, 9, 9, 6, 0, 9, 9, 8, 1, 4, 1, 5, 5, 8, 1, 4, 7, 5, 9, 0, 8, 9, 5, 5, 9, 9, 4, 8, 0, 5, 8, 3, 1, 6, 3, 5, 8, 8, 4, 7, 9, 7, 7, 4, 1, 9, 5, 5, 0, 4, 3, 8, 7, 7, 8, 5, 0, 4, 8, 7, 4, 4, 5, 2, 5, 0, 0, 5, 3, 6, 5, 4, 3, 0, 9, 1, 4, 9, 0, 2, 1, 7, 4, 4, 2, 0, 7, 4, 2, 7, 1, 3, 5, 7, 1, 7, 2, 5, 2, 1, 4, 1, 6, 0, 1, 0, 7, 4, 5, 5, 3, 6, 1, 9, 3, 7, 1, 6, 2, 5, 9, 9, 7, 1, 0, 5, 8, 0, 6, 9, 8, 8, 6, 9, 0, 8, 3, 2, 7, 3, 1, 9, 8, 3, 3, 7, 3, 8, 8, 2, 9, 7, 4, 5, 3, 5, 5, 0, 9, 7, 0, 6, 9, 5, 2, 2, 2, 9, 1, 4, 3, 6, 9, 6, 5, 2, 8, 6, 3, 1, 0, 5, 5, 6, 7, 1, 9, 7, 6, 1, 7, 8, 7, 4, 9, 1, 3, 1, 2, 6, 7, 2, 7, 3, 8, 0, 9, 0, 6, 3, 5, 5, 1, 8, 4, 6, 4, 5, 2, 5, 2, 3, 6, 7, 7, 3, 8, 6, 3, 0, 4, 3, 6, 7, 5, 6, 1, 3, 2, 9, 3, 6, 8, 9, 9, 3, 1, 4, 7, 2, 7, 4, 7, 7, 0, 7, 7, 2, 4, 7, 6, 1, 3, 5, 0, 0, 0, 1, 0, 6, 4, 3, 4, 4, 2, 1, 4, 6, 4, 7, 0, 4, 7, 1, 1, 1, 9

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9090, F1-score: 0.9089, Precision: 0.9094, Recall: 0.9090, AUROC: 0.9887


## **S=20 R=5:**



**Initialize SISA**


In [ ]:
S = 20
R = 5
epochs = 5

sisa_S20_R5 = SISA(S, R, epochs)


**download model from saved files:**

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = ''

# Download and extract models
sisa_S5_R5.download_and_extract_models_from_drive(file_id)


**load model from zip file:**

In [ ]:
sisa_S20_R5.extract_and_load_model('/content/models_S20_R5.zip')

**Unlearning:**

In [ ]:
sisa_S20_R5.unlearn(cifar10_train, 500)

0  --->  500
overlap:  {98, 259, 452, 400, 21}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1, @ slice: 1, @ epoch: 1, loss: 2.0960512161254883
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 1.6309154033660889
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 1.0076566934585571
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 0.5003425478935242
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 0.3150418996810913
0  --->  1000
overlap:  {98, 259, 452, 744, 910, 400, 595, 21, 569}
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 0.6189245581626892
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 0.4671448767185211
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.2284207046031952
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 0.09563322365283966
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 0.015315432101488113
0  --->  1500
overlap:  {259, 1411, 1029, 1417, 1420, 910, 400, 1041, 21, 1436, 1438, 1206, 569, 1090, 452, 1359, 595, 1493, 98, 744}
@ shard: 1, @ slice: 3, @ epoch: 1, loss: 0.4416049122810364
@ shard: 1, @ slice: 3, @ epoch: 2, loss: 0.17217221856117249
@ shard: 1, @ slice: 3, @ epoch: 3, loss: 0.09723613411188126


**performance using your proposed metric**

In [ ]:

print("unlearning time:",sisa_S20_R5.unlearning_time,"sec")
print("unlearning percent:",sisa_S20_R5.unlearning_percent*100,"%")

unlearning time: 806.345433473587 sec
unlearning percent: 100.0 %


**Test Data:**

In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Get a sample of 100 test examples
sample_indices = torch.randperm(len(cifar10_test))[:1000]
test_data = torch.utils.data.Subset(cifar10_test, sample_indices)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

Files already downloaded and verified



*  **weighted average aggregation:**


In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S20_R5.evaluate_with_weighted_aggregate(test_loader, sisa_S20_R5.aggregate_weighted_average, validation_loader)


all_preds= [8, 0, 0, 0, 1, 7, 4, 8, 2, 3, 7, 4, 8, 3, 6, 5, 7, 8, 1, 1, 5, 5, 6, 8, 9, 0, 9, 7, 8, 9, 8, 1, 1, 0, 6, 0, 9, 5, 1, 7, 4, 6, 7, 1, 9, 3, 4, 5, 8, 8, 2, 4, 8, 1, 4, 1, 9, 9, 6, 5, 2, 0, 5, 8, 0, 0, 7, 7, 3, 5, 9, 5, 0, 7, 6, 9, 7, 8, 4, 2, 8, 1, 4, 1, 3, 0, 0, 7, 9, 2, 9, 8, 9, 6, 8, 4, 5, 5, 8, 5, 6, 2, 1, 7, 3, 4, 0, 3, 9, 7, 6, 7, 0, 1, 6, 4, 7, 3, 1, 0, 9, 8, 0, 5, 7, 8, 9, 3, 9, 6, 6, 6, 7, 5, 2, 8, 4, 2, 4, 6, 8, 1, 2, 0, 1, 8, 8, 0, 6, 9, 9, 6, 7, 9, 2, 5, 8, 8, 3, 5, 5, 4, 3, 1, 8, 2, 5, 1, 2, 4, 0, 2, 9, 2, 3, 5, 0, 3, 9, 2, 1, 8, 9, 4, 6, 8, 1, 6, 6, 7, 1, 9, 9, 9, 8, 7, 7, 5, 4, 2, 7, 4, 9, 8, 4, 7, 0, 7, 8, 0, 5, 6, 1, 0, 1, 4, 9, 5, 9, 6, 3, 2, 6, 2, 9, 1, 6, 6, 5, 9, 1, 9, 1, 1, 9, 4, 5, 8, 6, 6, 3, 1, 7, 1, 9, 7, 4, 2, 4, 2, 2, 1, 3, 6, 6, 5, 7, 9, 4, 2, 9, 4, 9, 7, 1, 9, 6, 4, 6, 9, 5, 2, 3, 0, 1, 7, 4, 7, 4, 5, 8, 0, 0, 0, 2, 6, 9, 8, 7, 9, 2, 7, 5, 2, 6, 0, 0, 8, 1, 5, 8, 0, 7, 9, 2, 2, 3, 2, 6, 1, 5, 1, 5, 4, 9, 3, 7, 5, 6, 8, 9, 2, 2, 8, 9, 6, 8, 8, 0, 8

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9050, F1-score: 0.9044, Precision: 0.9059, Recall: 0.9050, AUROC: 0.9932



*  **average aggregation:**


In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S20_R5.evaluate(test_loader, sisa_S20_R5.aggregate_simple_average)


all_preds= [8, 0, 0, 0, 1, 7, 4, 8, 2, 3, 7, 4, 8, 3, 6, 5, 7, 8, 1, 1, 5, 5, 6, 8, 9, 0, 9, 7, 8, 9, 8, 1, 1, 0, 6, 0, 9, 5, 1, 7, 4, 6, 7, 1, 9, 3, 4, 5, 8, 8, 2, 4, 8, 1, 4, 1, 9, 9, 6, 5, 2, 0, 5, 8, 0, 0, 7, 7, 3, 5, 9, 5, 0, 7, 6, 9, 7, 8, 4, 2, 8, 1, 4, 1, 3, 0, 0, 7, 9, 2, 9, 8, 9, 6, 8, 4, 5, 5, 8, 5, 6, 2, 1, 7, 3, 4, 0, 3, 9, 7, 6, 7, 0, 1, 6, 4, 7, 3, 1, 0, 9, 8, 0, 5, 7, 8, 9, 3, 9, 6, 6, 6, 7, 5, 2, 8, 4, 2, 4, 6, 8, 1, 2, 0, 1, 8, 8, 0, 6, 9, 9, 6, 7, 9, 2, 5, 8, 8, 3, 5, 5, 4, 3, 1, 8, 2, 5, 1, 2, 4, 0, 2, 9, 2, 3, 5, 0, 3, 9, 2, 1, 8, 9, 4, 6, 8, 1, 6, 6, 7, 1, 9, 9, 9, 8, 7, 7, 5, 4, 2, 7, 4, 9, 8, 4, 7, 0, 7, 8, 0, 5, 6, 1, 0, 1, 4, 9, 5, 9, 6, 3, 2, 6, 2, 9, 1, 6, 6, 5, 9, 1, 9, 1, 1, 9, 4, 5, 8, 6, 6, 3, 1, 7, 1, 9, 7, 4, 2, 4, 2, 2, 1, 3, 6, 6, 5, 7, 9, 4, 2, 9, 4, 9, 7, 1, 9, 6, 4, 6, 9, 5, 2, 3, 0, 1, 7, 4, 7, 4, 5, 8, 0, 0, 0, 2, 6, 9, 8, 7, 9, 2, 7, 5, 2, 6, 0, 0, 8, 1, 5, 8, 0, 7, 9, 2, 2, 3, 2, 6, 1, 5, 1, 5, 4, 9, 3, 7, 5, 6, 8, 9, 2, 2, 8, 9, 6, 8, 8, 0, 8

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.9040, F1-score: 0.9034, Precision: 0.9049, Recall: 0.9040, AUROC: 0.9952



*  **majority vote aggregation:**


In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S20_R5.evaluate(test_loader, sisa_S20_R5.aggregate_majority_vote)

all_preds= [8, 0, 0, 0, 1, 7, 4, 8, 2, 3, 7, 4, 8, 3, 6, 5, 4, 8, 1, 1, 5, 5, 6, 8, 9, 0, 9, 7, 8, 9, 8, 1, 1, 0, 6, 0, 9, 5, 1, 7, 4, 6, 7, 1, 9, 3, 4, 5, 8, 8, 2, 4, 8, 1, 4, 1, 9, 9, 6, 5, 2, 0, 5, 8, 0, 0, 7, 7, 7, 5, 9, 5, 0, 7, 6, 9, 7, 8, 4, 2, 8, 1, 4, 1, 3, 0, 0, 7, 9, 2, 9, 8, 9, 6, 8, 4, 5, 5, 8, 5, 6, 2, 1, 7, 3, 4, 0, 3, 9, 7, 6, 7, 0, 1, 6, 4, 7, 3, 1, 0, 9, 8, 0, 5, 7, 8, 9, 3, 9, 6, 6, 6, 7, 5, 2, 8, 7, 2, 4, 6, 8, 1, 2, 0, 1, 8, 8, 0, 6, 9, 9, 6, 7, 9, 2, 5, 8, 8, 3, 5, 5, 4, 3, 1, 8, 2, 5, 1, 2, 4, 0, 3, 9, 2, 3, 5, 0, 3, 9, 2, 1, 8, 9, 4, 6, 8, 1, 6, 6, 7, 1, 9, 9, 9, 8, 7, 7, 5, 4, 2, 7, 4, 9, 8, 4, 7, 0, 7, 8, 0, 5, 6, 1, 0, 1, 4, 9, 5, 9, 6, 3, 2, 6, 2, 9, 1, 6, 6, 5, 9, 1, 9, 1, 1, 9, 4, 5, 8, 6, 6, 3, 1, 7, 1, 9, 7, 4, 2, 4, 2, 2, 1, 3, 6, 6, 5, 7, 9, 4, 2, 9, 4, 9, 7, 1, 9, 6, 4, 6, 9, 5, 2, 3, 0, 1, 7, 4, 7, 4, 5, 8, 0, 0, 0, 2, 6, 9, 8, 7, 9, 2, 7, 5, 2, 6, 0, 0, 8, 1, 5, 8, 0, 7, 9, 2, 2, 3, 2, 6, 1, 5, 1, 5, 4, 9, 3, 7, 5, 6, 8, 9, 2, 2, 8, 9, 6, 8, 8, 0, 8

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9010, F1-score: 0.9004, Precision: 0.9019, Recall: 0.9010, AUROC: 0.9916


## **S=20 R=10:**


**Initialize SISA**

In [ ]:
S = 20
R = 10
epochs = 5

sisa_S20_R10 = SISA(S, R, epochs)

**download model from saved files:**

In [ ]:

# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = ''

# Download and extract models
sisa_S20_R10.download_and_extract_models_from_drive(file_id)



**load model from zip file:**


In [ ]:

sisa_S20_R10.extract_and_load_model('/content/models_S20_R10.zip')


**Unlearning:**

In [ ]:
sisa_S20_R10.unlearn(cifar10_train, 500)

0  --->  250
overlap:  {29}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1, @ slice: 1, @ epoch: 1, loss: 2.2624213695526123
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 2.1769721508026123
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 1.873227596282959
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 1.4008113145828247
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 0.9496606588363647
0  --->  500
overlap:  {265, 29, 326}
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 1.452180027961731
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 1.0399341583251953
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.6954509019851685
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 0.4363495111465454
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 0.24891775846481323
0  --->  750
overlap:  {326, 265, 521, 553, 533, 29}
@ shard: 1, @ slice: 3, @ epoch: 1, loss: 0.8631155490875244
@ shard: 1, @ slice: 3, @ epoch: 2, loss: 0.4622341990470886
@ shard: 1, @ slice: 3, @ epoch: 3, loss: 0.34330520033836365
@ shard: 1, @ slice: 3, @ epoch: 4, loss: 0.18836824595928192
@ shard: 1, @ slice: 3, @ epoch: 5, loss: 0.0786587893


**performance using your proposed metric**


In [ ]:
print("unlearning time:",sisa_S20_R10.unlearning_time,"sec")
print("unlearning percent:",sisa_S20_R10.unlearning_percent*100,"%")

unlearning time: 891.0255763530731 sec
unlearning percent: 98.0 %



**Test Data:**



In [ ]:


cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Get a sample of 100 test examples
sample_indices = torch.randperm(len(cifar10_test))[:1000]
test_data = torch.utils.data.Subset(cifar10_test, sample_indices)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=False)

Files already downloaded and verified



*  **weighted average aggregation:**


In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S20_R10.evaluate_with_weighted_aggregate(test_loader, sisa_S20_R10.aggregate_weighted_average, validation_loader)


all_preds= [5, 2, 6, 5, 3, 3, 0, 8, 6, 1, 5, 9, 6, 2, 0, 2, 5, 7, 9, 3, 3, 6, 9, 5, 5, 0, 6, 5, 5, 1, 1, 0, 9, 6, 3, 6, 4, 5, 1, 2, 9, 7, 5, 4, 0, 9, 8, 7, 7, 4, 8, 4, 6, 0, 7, 1, 6, 1, 0, 5, 1, 3, 3, 4, 4, 2, 0, 4, 1, 4, 2, 7, 9, 0, 3, 3, 9, 7, 1, 3, 1, 7, 8, 2, 8, 8, 7, 3, 1, 7, 1, 3, 5, 3, 6, 5, 0, 8, 9, 8, 7, 4, 9, 0, 7, 6, 3, 7, 2, 6, 1, 7, 4, 1, 1, 2, 0, 2, 3, 5, 2, 1, 7, 8, 8, 8, 4, 6, 8, 7, 3, 1, 1, 0, 1, 3, 4, 1, 0, 0, 1, 6, 0, 3, 5, 0, 1, 3, 3, 4, 0, 1, 1, 8, 9, 5, 3, 5, 0, 5, 6, 3, 1, 4, 1, 1, 7, 2, 5, 1, 7, 2, 6, 1, 5, 4, 6, 1, 5, 0, 0, 5, 0, 7, 4, 7, 4, 7, 2, 9, 1, 9, 5, 0, 9, 6, 8, 1, 7, 9, 2, 6, 4, 5, 1, 1, 5, 3, 9, 5, 4, 0, 9, 2, 8, 0, 3, 9, 2, 6, 3, 0, 2, 1, 4, 4, 6, 4, 4, 0, 5, 4, 1, 7, 1, 5, 3, 3, 4, 3, 1, 6, 2, 9, 9, 8, 9, 1, 5, 5, 9, 8, 2, 2, 2, 4, 3, 1, 2, 2, 1, 3, 1, 0, 0, 9, 6, 0, 4, 0, 6, 6, 1, 0, 1, 9, 3, 5, 0, 0, 4, 7, 8, 5, 8, 7, 3, 0, 1, 2, 8, 0, 8, 9, 9, 2, 3, 4, 2, 4, 1, 6, 1, 0, 6, 3, 7, 8, 5, 1, 7, 1, 9, 9, 2, 0, 5, 2, 1, 7, 8, 9, 1, 5, 3, 9, 3, 0, 7, 5

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9070, F1-score: 0.9068, Precision: 0.9073, Recall: 0.9070, AUROC: 0.9920



*  **average aggregation:**


In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S20_R10.evaluate(test_loader, sisa_S20_R10.aggregate_simple_average)

all_preds= [5, 2, 6, 5, 3, 3, 0, 8, 6, 1, 5, 9, 6, 2, 0, 2, 5, 7, 9, 3, 3, 6, 9, 5, 5, 0, 6, 5, 5, 1, 1, 0, 9, 6, 3, 6, 4, 5, 1, 2, 9, 7, 5, 4, 0, 9, 8, 7, 7, 4, 8, 4, 6, 0, 7, 1, 6, 1, 0, 5, 1, 3, 3, 4, 4, 2, 0, 4, 1, 4, 2, 7, 9, 0, 3, 3, 9, 7, 1, 3, 1, 7, 8, 2, 8, 8, 7, 3, 1, 7, 1, 3, 5, 3, 6, 5, 0, 8, 9, 8, 7, 4, 9, 0, 7, 6, 3, 7, 2, 6, 1, 7, 4, 1, 1, 2, 0, 2, 3, 5, 2, 1, 7, 8, 8, 8, 4, 6, 8, 7, 3, 1, 1, 0, 1, 3, 4, 1, 0, 0, 1, 6, 0, 3, 5, 0, 1, 3, 3, 4, 0, 1, 1, 8, 9, 5, 3, 5, 0, 5, 6, 3, 1, 4, 1, 1, 7, 2, 5, 1, 7, 2, 6, 1, 5, 4, 6, 1, 5, 0, 0, 5, 0, 7, 4, 7, 4, 7, 2, 9, 1, 9, 5, 0, 9, 6, 8, 1, 7, 9, 2, 6, 4, 5, 1, 1, 5, 3, 9, 5, 4, 0, 9, 2, 8, 0, 3, 9, 2, 6, 3, 0, 2, 1, 4, 4, 6, 4, 4, 0, 5, 4, 1, 7, 1, 5, 3, 3, 4, 3, 1, 6, 2, 9, 9, 8, 9, 1, 5, 5, 9, 8, 2, 2, 2, 4, 3, 1, 2, 2, 1, 3, 1, 0, 0, 9, 6, 0, 4, 0, 6, 6, 1, 0, 1, 9, 3, 5, 0, 0, 4, 7, 8, 5, 8, 7, 3, 0, 1, 2, 8, 0, 8, 9, 9, 2, 3, 4, 2, 4, 1, 6, 1, 0, 6, 3, 7, 8, 5, 1, 7, 1, 9, 9, 2, 0, 5, 2, 1, 7, 8, 9, 1, 5, 3, 9, 3, 0, 7, 5

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9050, F1-score: 0.9048, Precision: 0.9055, Recall: 0.9050, AUROC: 0.9937



*  **majority vote aggregation:**


In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S20_R10.evaluate(test_loader, sisa_S20_R10.aggregate_majority_vote)

all_preds= [5, 2, 6, 5, 1, 3, 0, 8, 6, 1, 2, 9, 6, 2, 0, 2, 5, 7, 9, 3, 3, 6, 9, 5, 5, 0, 6, 5, 5, 1, 1, 0, 9, 6, 3, 6, 4, 5, 1, 2, 9, 7, 5, 4, 0, 9, 8, 7, 7, 4, 8, 4, 6, 0, 7, 1, 6, 1, 0, 5, 1, 3, 3, 4, 4, 2, 0, 7, 1, 4, 2, 7, 9, 0, 3, 3, 9, 7, 1, 3, 1, 7, 8, 2, 8, 8, 7, 3, 1, 7, 1, 3, 5, 3, 6, 5, 0, 8, 9, 8, 7, 4, 9, 0, 7, 6, 3, 7, 2, 6, 1, 7, 4, 1, 1, 2, 0, 2, 3, 5, 2, 1, 7, 8, 8, 8, 4, 6, 8, 7, 5, 1, 1, 0, 1, 3, 4, 1, 0, 0, 1, 3, 0, 3, 5, 0, 1, 3, 3, 4, 0, 1, 1, 8, 9, 5, 3, 5, 0, 5, 6, 3, 1, 4, 1, 1, 7, 2, 5, 1, 7, 2, 6, 1, 5, 4, 6, 1, 5, 0, 0, 7, 0, 7, 4, 7, 4, 7, 2, 9, 1, 9, 5, 0, 9, 6, 8, 1, 7, 9, 2, 6, 4, 5, 1, 1, 5, 3, 9, 5, 4, 0, 9, 2, 8, 0, 3, 9, 2, 6, 3, 0, 2, 1, 4, 4, 6, 4, 4, 0, 5, 4, 1, 7, 1, 3, 3, 3, 4, 3, 1, 6, 2, 9, 0, 8, 9, 1, 5, 5, 9, 8, 2, 2, 2, 4, 3, 1, 2, 2, 1, 3, 1, 0, 0, 9, 6, 0, 4, 0, 6, 6, 1, 0, 1, 9, 3, 5, 0, 0, 4, 7, 8, 5, 8, 7, 3, 0, 1, 2, 8, 0, 8, 9, 9, 2, 3, 4, 2, 4, 1, 6, 1, 0, 6, 3, 7, 8, 5, 1, 7, 1, 9, 9, 2, 0, 5, 2, 1, 7, 8, 9, 1, 5, 3, 9, 3, 0, 7, 5

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.9030, F1-score: 0.9027, Precision: 0.9035, Recall: 0.9030, AUROC: 0.9895


## **S=20 R=20:**

**Initialize SISA**

In [ ]:
S = 20
R = 20
epochs = 5

sisa_S20_R20 = SISA(S, R, epochs)


**download model from saved files:**

In [ ]:

# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = ''

# Download and extract models
sisa_S20_R20.download_and_extract_models_from_drive(file_id)



**load model from zip file:**


In [ ]:

sisa_S20_R20.extract_and_load_model('/content/models_S5_R5.zip')



**Unlearning:**


In [ ]:
sisa_S20_R20.unlearn(cifar10_train, 500)

0  --->  125
overlap:  {46, 38}


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1, @ slice: 1, @ epoch: 1, loss: 2.3075990676879883
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 2.1429266929626465
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 2.159071922302246
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 1.8407206535339355
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 1.6852558851242065
0  --->  250
overlap:  {46, 210, 38}
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 2.037546396255493
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 1.8974705934524536
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 1.6019192934036255
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 1.427983045578003
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 1.1592087745666504
0  --->  375
overlap:  {46, 210, 38, 351}
@ shard: 1, @ slice: 3, @ epoch: 1, loss: 1.428443431854248
@ shard: 1, @ slice: 3, @ epoch: 2, loss: 1.4370172023773193
@ shard: 1, @ slice: 3, @ epoch: 3, loss: 1.1489065885543823
@ shard: 1, @ slice: 3, @ epoch: 4, loss: 0.9786390662193298
@ shard: 1, @ slice: 3, @ epoch: 5, loss: 0.6971507668495178
0  --->  5


**performance using your proposed metric**


In [ ]:

print("unlearning time:",sisa_S20_R20.unlearning_time,"sec")
print("unlearning percent:",sisa_S20_R20.unlearning_percent*100,"%")


unlearning time: 1075.838923215866 sec
unlearning percent: 97.25 %



**Test Data:**


In [ ]:

cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)


Files already downloaded and verified



*  **weighted average aggregation:**


In [ ]:

# Get a sample of 100 validation examples
validation_indices = torch.randperm(len(cifar10_train))[:1000]
validation_data = torch.utils.data.Subset(cifar10_train, validation_indices)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=False)

accuracy, f1, precision, recall, auroc = sisa_S20_R20.evaluate_with_weighted_aggregate(test_loader, sisa_S20_R20.aggregate_weighted_average, validation_loader)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 3, 0, 9, 6, 6, 5, 4, 3, 9, 3, 4, 9, 9, 5, 0, 6, 7, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 3, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 7, 2, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 2, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 3, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 7, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 3, 9, 0, 4, 3, 6, 5, 1, 3, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 9, 2, 1, 2, 8, 6, 8, 0, 4, 3, 7, 7, 5, 5, 3, 3, 2, 3, 7, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.8942, F1-score: 0.8947, Precision: 0.8979, Recall: 0.8942, AUROC: 0.9927



*  **average aggregation:**


In [ ]:
accuracy, f1, precision, recall, auroc = sisa_S20_R20.evaluate(test_loader, sisa_S20_R20.aggregate_simple_average)

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 3, 0, 9, 6, 6, 5, 4, 3, 9, 3, 4, 9, 9, 5, 0, 6, 7, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 3, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 7, 2, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 2, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 3, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 3, 0, 0, 5, 2, 2, 7, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 3, 9, 3, 4, 3, 6, 5, 1, 3, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 8, 2, 1, 2, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 7, 1, 7, 5, 4, 6

In [ ]:
print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')

Accuracy: 0.8952, F1-score: 0.8957, Precision: 0.8991, Recall: 0.8952, AUROC: 0.9938



*  **majority vote aggregation:**


In [ ]:

accuracy, f1, precision, recall, auroc = sisa_S20_R20.evaluate(test_loader, sisa_S20_R20.aggregate_majority_vote)


all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 4, 2, 4, 0, 9, 6, 6, 5, 4, 3, 9, 3, 4, 9, 9, 5, 0, 6, 7, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 3, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 7, 3, 7, 8, 3, 1, 2, 8, 2, 8, 3, 3, 0, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 2, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 3, 6, 0, 4, 8, 4, 5, 8, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 7, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 3, 9, 4, 3, 9, 0, 4, 3, 6, 5, 1, 3, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 3, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 0, 9, 2, 1, 2, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 3, 2, 3, 7, 1, 7, 5, 4, 6

In [ ]:

print(f'Accuracy: {accuracy:.4f}, F1-score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, AUROC: {auroc:.4f}')


Accuracy: 0.8885, F1-score: 0.8889, Precision: 0.8928, Recall: 0.8885, AUROC: 0.9903


# **Simulation Question 3.**
Write a function that takes losses of two different datasets of your
trained(but not unlearned) model,Computes cross-validation score of a Logistic Regressionbased Membership Inference Attack. Now, input the function with the losses of the ’forget set’
and 500 randomly chosen data of the test set of the trained model and report the score. Do the
same for the unlearned model. What did you expect the result to be for a perfectly unlearned
model? How did the SISA Algorithm perform?

**Step 0:**


In [22]:
# transform to include resizing to 224x224
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [23]:
cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

100%|██████████| 170498071/170498071 [00:03<00:00, 49594127.85it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [24]:
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


In [25]:
S = 5
R = 5
epochs = 5

sisa_3 = SISA(S, R, epochs)

In [26]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = '1YQwzzpRoQuNQ1ubKp5LDbthBdSnMVIoP'

# Download and extract models
sisa_3.download_and_extract_models_from_drive(file_id)

Downloaded models_S5_R5.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 61.9MB/s]


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


**Step 1: Cross-validation function**


In [27]:
def cross_validate_loss_vectors(loss_in, loss_out):
    # Create labels
    y_in = np.ones(len(loss_in))
    y_out = np.zeros(len(loss_out))

    # Combine the data
    X = np.concatenate([loss_in, loss_out])
    y = np.concatenate([y_in, y_out])

    # Reshape X for logistic regression
    X = X.reshape(-1, 1)

    # Perform cross-validation with Logistic Regression
    clf = LogisticRegressionCV(cv=5, random_state=0).fit(X, y)
    scores = cross_val_score(clf, X, y, cv=5)

    return np.mean(scores), np.std(scores)

**Step 2: Create datasets with "in" and "out" labels**


In [28]:
# Get 500 samples from the training set (in) and 500 from the test set (out)
train_indices = torch.randperm(len(cifar10_train))[:500]
test_indices = torch.randperm(len(cifar10_test))[:500]

train_subset = Subset(cifar10_train, train_indices)
test_subset = Subset(cifar10_test, test_indices)

train_loader = DataLoader(train_subset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=64, shuffle=False)

**Step 3: Calculate losses for "in" and "out" datasets**


In [29]:
# Calculate losses for in and out datasets
losses_in = sisa_3.calculate_losses(train_loader)
losses_out = sisa_3.calculate_losses(test_loader)

In [30]:
# Perform cross-validation before unlearning
mean_score_before, std_score_before = cross_validate_loss_vectors(losses_in, losses_out)
print(f'Cross-validation mean score before unlearning: {mean_score_before}, std: {std_score_before}')

Cross-validation mean score before unlearning: 0.529, std: 0.016552945357246864


**Step 4: Perform unlearning**


In [31]:
# Assuming we have an unlearn function already implemented in SISA class
# Randomly choose 500 samples to forget from the train_subset
forget_indices = np.array(train_indices)

sisa_3.unlearn_with_indices(cifar10_train, forget_indices)

[  453 36779 44157 34753 41930 13253 28431 24260 18637 14920 49332 24128
 11556 45731 49303 17417 36254 44819 46053 37900 32305 34817 39844 29995
  8413 49884 21798  8299 18956 32744 11101 44554 26174 20797 18592 18336
  3552 27844 37052  6040  4272 15653 19468  1101 18055 22987 12461 26086
   117  9985 39780 43753 41446 46662 17655 39852 40671 39980 17469 39701
 28103 21162 11965 48172 17422 45360 14585 42637  7961 27293 29218  4566
 20969 12587 36839 31852 41913  1798 45329 13157 20386 24443 39819  5194
 15888 39182 46240 20653 23635 47902 35490 12272 22590 37295 20886 47693
 30609 37136 48058 12011  1566 28876 15421 18983 13338 37290 35268 44117
  9273 20754 45161 44066 44639 39601 35401  6332 48357 38210   212 41004
  7283 32825 20632 34952 47207 44039 31667 24528 23714  1518 40893 10545
 30079 18379  8992 42572 32213 36782 31264 16693 40256 35024  8949 22442
 26240 38294 29571 44609 25614  5226 23185 31238 39030  3850 22630 35267
 18947 15735 47620 37813 14623 14547 44920 43471 25

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1, @ slice: 1, @ epoch: 1, loss: 2.0725693702697754
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 0.9068943858146667
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 2.552072048187256
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 1.0274736881256104
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 1.1198022365570068
0  --->  4000
overlap:  {3841, 3333, 1798, 2824, 3850, 1675, 2955, 271, 3983, 3221, 1687, 3356, 1566, 415, 825, 3130, 2113, 453, 3656, 1101, 212, 3552, 3297, 3940, 1518, 367, 1780, 117}
idx_slice != 1
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 0.6295835375785828
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 1.5634175539016724
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 2.730760097503662
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 3.430098056793213
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 1.204058051109314
0  --->  6000
overlap:  {3841, 3333, 1798, 2824, 3850, 1675, 2955, 271, 3983, 3221, 1687, 5400, 3356, 4126, 1566, 415, 4901, 4007, 5546, 4272, 4920, 825, 3130, 5050, 2113, 453, 5573, 3656, 5194

**Step 5: Calculate losses after unlearning**

In [32]:
# Calculate losses for in and out datasets after unlearning
losses_in_after = sisa_3.calculate_losses(train_loader)
losses_out_after = sisa_3.calculate_losses(test_loader)

# Perform cross-validation after unlearning
mean_score_after, std_score_after = cross_validate_loss_vectors(losses_in_after, losses_out_after)
print(f'Cross-validation mean score after unlearning: {mean_score_after}, std: {std_score_after}')


Cross-validation mean score after unlearning: 0.517, std: 0.012083045973594582


# **Add On. Simulation Question 1.**
Randomly select 500 data points from a specific class in
the training set. For each selected data point, turn a 3x3 block of pixels to black, with the
position of the block chosen randomly. Train your best model of the previous questions, using
the poisoned dataset. Evaluate the model’s performance, using all the same metrics as before,
on clean test data to ensure general performance is not significantly degraded. Also, calculate
the attack success rate (ASR) as the percentage of test samples misclassified as the target class.

In [33]:
# Step 1: Select 500 data points from a specific class in the training set
def select_and_poison_data_inplace(dataset, target_class, num_samples=500, block_size=3):
    class_indices = [i for i in range(len(dataset)) if dataset.targets[i] == target_class]
    selected_indices = random.sample(class_indices, num_samples)
    block_size = int(224 / 32) * block_size
    if block_size != 0:
      for idx in selected_indices:
          #print(dataset.targets[idx])
          img = dataset.data[idx]
          img = np.copy(img)  # To avoid modifying the original image
          # Randomly select a position for the 3x3 block
          x = random.randint(0, img.shape[0] - block_size)
          y = random.randint(0, img.shape[1] - block_size)
          img[x:x+block_size, y:y+block_size, :] = 0  # Turn the block to black
          dataset.data[idx] = img

    return selected_indices

In [34]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

cifar10_train = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

Files already downloaded and verified


In [35]:
# Step 2: Poison the data
poisoned_indices = select_and_poison_data_inplace(cifar10_train, target_class=0, num_samples=500, block_size=3)

In [36]:
print(poisoned_indices)

[32971, 35434, 37049, 42786, 35761, 40355, 31284, 31521, 10578, 47253, 11237, 40193, 20763, 35554, 12668, 44662, 41352, 8898, 27007, 25712, 22579, 41774, 17316, 8774, 45288, 49761, 18192, 29659, 35683, 29844, 31412, 1878, 38963, 31087, 28400, 9905, 34019, 17166, 25220, 36337, 2932, 33662, 47805, 32538, 40796, 38937, 46897, 12181, 7991, 15949, 25489, 5956, 14698, 8660, 39301, 37517, 14488, 48416, 21248, 32885, 18614, 29307, 29173, 28033, 11783, 18218, 17396, 25787, 13583, 46922, 43907, 37066, 1123, 16371, 41900, 28126, 39826, 13194, 27552, 14213, 28828, 32613, 39530, 39793, 33831, 39223, 23566, 14153, 38052, 45262, 8842, 38970, 4524, 30520, 22249, 20790, 8889, 43965, 29864, 17458, 27686, 20902, 30667, 46287, 13801, 23254, 5163, 1926, 44468, 48557, 29880, 18271, 47550, 27813, 42372, 25762, 9890, 36323, 39380, 5549, 5803, 7327, 49245, 7925, 8102, 37664, 19503, 40420, 28380, 10297, 39260, 16763, 27048, 598, 16305, 4162, 24567, 8858, 10326, 12802, 6682, 17884, 12355, 33127, 24169, 18018, 17

In [37]:
# Initialize SISA
S = 5
R = 5
epochs = 5
poisone_sisa = SISA(S, R, epochs)

In [ ]:
poisone_sisa.extract_and_load_model('/content/models_S5_R5.zip')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


In [38]:
# Train the model
poisone_sisa.train(cifar10_train)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1  @ slice: 1 @ epoch: 0   loss= 0.9146019220352173
@ shard: 1  @ slice: 1 @ epoch: 1   loss= 0.8537952899932861
@ shard: 1  @ slice: 1 @ epoch: 2   loss= 0.10981471091508865
@ shard: 1  @ slice: 1 @ epoch: 3   loss= 0.0836019441485405
@ shard: 1  @ slice: 1 @ epoch: 4   loss= 0.017733069136738777
@ shard: 1  @ slice: 2 @ epoch: 0   loss= 0.48237401247024536
@ shard: 1  @ slice: 2 @ epoch: 1   loss= 0.23053498566150665
@ shard: 1  @ slice: 2 @ epoch: 2   loss= 0.37628617882728577
@ shard: 1  @ slice: 2 @ epoch: 3   loss= 0.40827205777168274
@ shard: 1  @ slice: 2 @ epoch: 4   loss= 0.023045912384986877
@ shard: 1  @ slice: 3 @ epoch: 0   loss= 0.20134596526622772
@ shard: 1  @ slice: 3 @ epoch: 1   loss= 0.09379276633262634
@ shard: 1  @ slice: 3 @ epoch: 2   loss= 0.11052864044904709
@ shard: 1  @ slice: 3 @ epoch: 3   loss= 0.038163866847753525
@ shard: 1  @ slice: 3 @ epoch: 4   loss= 0.02443019300699234
@ shard: 1  @ slice: 4 @ epoch: 0   loss= 1.0838900804519653
@ shard: 

***UPLOADING:***

In [ ]:
poisone_sisa.upload_models_to_drive()

Uploaded file with ID 1HdZyTSCZiA8sycdg-RAb3oJi6XgDNCcp


'1HdZyTSCZiA8sycdg-RAb3oJi6XgDNCcp'

In [ ]:
# Google Drive file ID of the saved model zip file (replace this with your actual file ID)
file_id = ''

# Download and extract models
poisone_sisa.download_and_extract_models_from_drive(file_id)

Downloaded models_S5_R5.zip from Google Drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loading shard: 1
loading shard: 2
loading shard: 3
loading shard: 4
loading shard: 5


In [39]:
# Step 4: Evaluate the model's performance on clean test data
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


In [ ]:
# Get a sample of 1000 test examples from class 3
class_3_indices = [i for i, (_, label) in enumerate(cifar10_test) if label == 0]
sample_indices = torch.randperm(len(class_3_indices))[:1000]
sampled_class_3_indices = [class_3_indices[i] for i in sample_indices]
test_data = torch.utils.data.Subset(cifar10_test, class_3_indices)

test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

In [40]:
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

In [41]:
accuracy, f1, precision, recall, auroc = poisone_sisa.evaluate(test_loader, poisone_sisa.aggregate_majority_vote)

print(f'Accuracy: {accuracy}, F1-score: {f1}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 3, 7, 5, 6, 3, 6, 2, 1, 2, 2, 7, 2, 6, 8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 3, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 6, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 3, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 0, 5, 9, 8, 0, 7, 9, 8, 2, 7, 5, 9, 4, 3, 9, 0, 4, 5, 6, 5, 1, 5, 8, 8, 0, 4, 9, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 8, 3, 0, 0, 9, 8, 1, 5, 3, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 2, 4, 3, 9, 0, 6, 1, 0, 9, 1, 8, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 9, 4, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [42]:
# Step 5: Calculate the Attack Success Rate (ASR)
def calculate_asr(sisa, dataset, num_samples, target_class=0):
    # Prepare poisoned images from other classes
    poisoned_datasets = []

    for i in range(1, 10):
        if i != target_class:
            poisoned_indices = select_and_poison_data_inplace(dataset, target_class=i, num_samples=num_samples, block_size=3)
            poisoned_subset = Subset(dataset, poisoned_indices)
            poisoned_datasets.append(poisoned_subset)


    # Concatenate all poisoned subsets into one dataset
    poisoned_dataset = ConcatDataset(poisoned_datasets)
    poisoned_loader = DataLoader(poisoned_dataset, batch_size=64, shuffle=False)

    aggregated_predictions, _ = sisa.aggregate_simple_average(poisoned_loader)
    #accuracy, f1, precision, recall, auroc = poisone_sisa.evaluate(poisoned_loader, poisone_sisa.aggregate_simple_average)



    print(aggregated_predictions)

    misclassified_count = (aggregated_predictions == target_class).sum().item()
    total_count = len(poisoned_dataset)

    return misclassified_count / total_count

In [43]:
asr = calculate_asr(poisone_sisa, cifar10_test, 100, 0)
print(f'Attack Success Rate (ASR): {asr * 100:.2f}%')

tensor([9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 9, 0, 0, 0, 0, 0, 0, 1, 0, 9, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 9, 0, 0, 0, 0,
        0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,

# **Add On. Simulation Question 2.**
 Unlearn the same 500 data, and evaluate the model’s
performance, using all the same metrics ass before, on clean test data to ensure general performance is not significantly degraded. Also, calculate the attack success rate (ASR) as the
percentage of test samples misclassified as the target class.

In [44]:
poisone_sisa.unlearn_with_indices(cifar10_train, poisoned_indices)

[32971, 35434, 37049, 42786, 35761, 40355, 31284, 31521, 10578, 47253, 11237, 40193, 20763, 35554, 12668, 44662, 41352, 8898, 27007, 25712, 22579, 41774, 17316, 8774, 45288, 49761, 18192, 29659, 35683, 29844, 31412, 1878, 38963, 31087, 28400, 9905, 34019, 17166, 25220, 36337, 2932, 33662, 47805, 32538, 40796, 38937, 46897, 12181, 7991, 15949, 25489, 5956, 14698, 8660, 39301, 37517, 14488, 48416, 21248, 32885, 18614, 29307, 29173, 28033, 11783, 18218, 17396, 25787, 13583, 46922, 43907, 37066, 1123, 16371, 41900, 28126, 39826, 13194, 27552, 14213, 28828, 32613, 39530, 39793, 33831, 39223, 23566, 14153, 38052, 45262, 8842, 38970, 4524, 30520, 22249, 20790, 8889, 43965, 29864, 17458, 27686, 20902, 30667, 46287, 13801, 23254, 5163, 1926, 44468, 48557, 29880, 18271, 47550, 27813, 42372, 25762, 9890, 36323, 39380, 5549, 5803, 7327, 49245, 7925, 8102, 37664, 19503, 40420, 28380, 10297, 39260, 16763, 27048, 598, 16305, 4162, 24567, 8858, 10326, 12802, 6682, 17884, 12355, 33127, 24169, 18018, 17

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


@ shard: 1, @ slice: 1, @ epoch: 1, loss: 0.8526992797851562
@ shard: 1, @ slice: 1, @ epoch: 2, loss: 0.5581055879592896
@ shard: 1, @ slice: 1, @ epoch: 3, loss: 0.1648118495941162
@ shard: 1, @ slice: 1, @ epoch: 4, loss: 0.09996895492076874
@ shard: 1, @ slice: 1, @ epoch: 5, loss: 0.06453915685415268
0  --->  4000
overlap:  {1926, 2950, 2824, 3344, 401, 3347, 405, 2714, 29, 3113, 2473, 3884, 940, 1329, 1593, 700, 3906, 1477, 199, 2504, 843, 1878, 598, 1632, 1123, 2659, 3815, 3304, 752, 2932, 3831, 2680}
idx_slice != 1
@ shard: 1, @ slice: 2, @ epoch: 1, loss: 0.48827728629112244
@ shard: 1, @ slice: 2, @ epoch: 2, loss: 0.31964847445487976
@ shard: 1, @ slice: 2, @ epoch: 3, loss: 0.036130040884017944
@ shard: 1, @ slice: 2, @ epoch: 4, loss: 0.022406786680221558
@ shard: 1, @ slice: 2, @ epoch: 5, loss: 0.0059235580265522
0  --->  6000
overlap:  {4097, 1926, 2950, 2824, 3344, 401, 3347, 405, 4373, 2714, 29, 3113, 2473, 5163, 4524, 5549, 5803, 3884, 940, 1329, 1593, 700, 4162, 390

In [45]:
# Step 4: Evaluate the model's performance on clean test data
cifar10_test = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

Files already downloaded and verified


In [46]:
test_loader = torch.utils.data.DataLoader(cifar10_test, batch_size=64, shuffle=False)

In [47]:
accuracy, f1, precision, recall, auroc = poisone_sisa.evaluate(test_loader, poisone_sisa.aggregate_majority_vote)

print(f'Accuracy: {accuracy}, F1-score: {f1}, Precision: {precision}, Recall: {recall}, AUROC: {auroc}')

all_preds= [3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 3, 4, 9, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 3, 3, 7, 5, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 5, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 6, 7, 8, 3, 1, 2, 8, 2, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 5, 8, 7, 6, 5, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 3, 4, 2, 1, 6, 6, 0, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 3, 3, 8, 6, 3, 2, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 3, 9, 6, 4, 3, 6, 5, 1, 3, 8, 8, 0, 4, 0, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 7, 8, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 3, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 9, 2, 9, 4, 8, 6, 0, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 7, 5, 4, 6

In [50]:
# Step 5: Calculate the Attack Success Rate (ASR)

asr_unlearn = calculate_asr(poisone_sisa, cifar10_test, 100, 0)
print(f'Attack Success Rate (ASR): {asr_unlearn * 100:.2f}%')

tensor([9, 3, 9, 0, 1, 9, 9, 9, 9, 3, 0, 9, 9, 9, 9, 9, 9, 1, 3, 9, 9, 8, 9, 9, 
        1, 9, 9, 9, 9, 3, 9, 9, 0, 9, 9, 9, 9, 3, 9, 9, 3, 9, 9, 9, 3, 9, 8, 9, 
        3, 9, 1, 9, 3, 0, 3, 9, 9, 3, 9, 9, 9, 9, 9, 1, 9, 9, 1, 9, 3, 9, 9, 3, 
        9, 3, 9, 9, 1, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 0, 9, 3, 9, 9, 3, 9, 
        2, 9, 3, 3, 3, 0, 3, 3, 6, 8, 3, 3, 2, 2, 3, 8, 9, 2, 2, 3, 9, 3, 2, 3, 
        8, 0, 8, 8, 3, 3, 3, 2, 3, 3, 2, 3, 2, 3, 3, 0, 2, 2, 3, 0, 3, 3, 2, 8, 
        3, 3, 2, 3, 2, 3, 0, 8, 2, 3, 3, 2, 3, 2, 0, 3, 2, 3, 3, 0, 3, 2, 3, 3, 
        2, 0, 2, 3, 0, 0, 3, 3, 3, 0, 3, 9, 2, 3, 2, 3, 0, 3, 3, 0, 3, 9, 0, 2, 
        8, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 3, 2, 8, 2, 3, 3, 3, 3, 3, 3, 3, 8, 9, 
        3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 0, 3, 2, 0, 
        3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 8, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
        3, 3, 3, 3, 3, 3, 8, 3, 3, 3, 3, 3, 2, 3, 9, 3, 3, 3, 3, 3, 3, 3, 3, 3, 
        3, 3, 3, 0, 3, 4, 3,

# **Simulation Question 4.**
Use 80 percent of the CIFAR-10 training data to train your model.
This will serve as your baseline model.

In [5]:
!pip install gdown
!pip install pydrive2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
from model import CIFAR10Classifier  # Importing the model
import os
import shutil
import zipfile
import gdown
# Check if CUDA is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Augmentation and Normalization
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
# Load CIFAR-10 Dataset

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# Initialize the model, criterion, and optimizer
model = CIFAR10Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

# Train the model
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Train the model
epochs = 30
best_val_acc = 0.0

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_acc = 100 * correct / total
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}, Val Loss: {val_loss / len(val_loader)}, Val Accuracy: {val_acc}%')

# Function to save the model
def zip_model(local_path, zip_name):
    import zipfile
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        zipf.write(local_path)

# Define functions for Google Drive operations
def upload_to_drive(local_path, drive_path):
    from google.colab import drive
    drive.mount('/content/drive')
    shutil.move(local_path, drive_path)

def download_from_drive(drive_path, local_path):
    from google.colab import drive
    drive.mount('/content/drive')
    shutil.copy(drive_path, local_path)

def unzip_model(zip_name, extract_path):
    import zipfile
    with zipfile.ZipFile(zip_name, 'r') as zipf:
        zipf.extractall(extract_path)

# Function to save the model
def save_model(model, path='cifar10_classifier.pth'):
    torch.save(model.state_dict(), path)
    print(f'Model saved to {path}')

# Function to load the model
def load_model(model, path='cifar10_classifier.pth'):
    model.load_state_dict(torch.load(path))
    model.to(device)
    model.eval()
    print(f'Model loaded from {path}')
    return model
# Save and zip the model
save_model(model, 'cifar10_classifier.pth')
zip_model('cifar10_classifier.pth', 'cifar10_classifier.zip')
# Upload the model to Google Drive
upload_to_drive('cifar10_classifier.zip', '/content/drive/MyDrive/cifar10_classifier.zip')

# Download the model from Google Drive and unzip it
download_from_drive('/content/drive/MyDrive/cifar10_classifier.zip', 'cifar10_classifier.zip')
unzip_model('cifar10_classifier.zip', './')
# Load the model
model = CIFAR10Classifier().to(device)
model = load_model(model, 'cifar10_classifier.pth')
# Evaluate the model on the test set
test_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Loss: {test_loss / len(test_loader)}')
print(f'Test Accuracy: {100 * correct / total}%')

Files already downloaded and verified 
Files already downloaded and verified 
Epoch 1/30, Loss: 1.8607441967010498, Val Loss: 1.5743849763445035, Val Accuracy: 44.77% 
Epoch 2/30, Loss: 1.6487779710769654, Val Loss: 1.4562632635140875, Val Accuracy: 49.56% 
Epoch 3/30, Loss: 1.5708419103622437, Val Loss: 1.3561321474184656, Val Accuracy: 52.1% 
Epoch 4/30, Loss: 1.513230711555481, Val Loss: 1.2891023227363636, Val Accuracy: 55.67% 
Epoch 5/30, Loss: 1.4894245246887208, Val Loss: 1.2699495071818114, Val Accuracy: 55.89% 
Epoch 6/30, Loss: 1.4545565574645996, Val Loss: 1.2337484617901455, Val Accuracy: 57.47% 
Epoch 7/30, Loss: 1.4351855136871339, Val Loss: 1.2241097195133281, Val Accuracy: 57.38% 
Epoch 8/30, Loss: 1.4069478324890137, Val Loss: 1.1770640175054028, Val Accuracy: 58.75% 
Epoch 9/30, Loss: 1.394394047164917, Val Loss: 1.2102323595885258, Val Accuracy: 58.44% 
Epoch 10/30, Loss: 1.389037771987915, Val Loss: 1.1417351084150327, Val Accuracy: 59.94% 
Epoch 11/30, Loss: 1.3370

# **Simulation Question 5.**
Train your baseline model with privacy enhancements. This is your
modified model. Ensure that the test accuracy difference between your baseline model and the
modified model is less than 15


In [7]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from model import CIFAR10Classifier  # Importing the model
import os
import shutil
import zipfile
import gdown

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Data Augmentation and Normalization
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 Dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Initialize the network
net = CIFAR10Classifier().to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(trainloader), epochs=50)

# Gradient Clipping and Noise Functions
def clip_gradients(parameters, max_norm):
    nn.utils.clip_grad_norm_(parameters, max_norm)

def add_noise_to_gradients(parameters, noise_multiplier, max_grad_norm):
    for p in parameters:
        if p.grad is not None:
            noise = torch.randn_like(p.grad) * noise_multiplier * max_grad_norm
            p.grad.data.add_(noise)

# Training loop
noise_multiplier = 0.1
max_grad_norm = 1.0

for epoch in range(60):  # Increase the number of epochs for better training
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        # Clip gradients
        clip_gradients(net.parameters(), max_grad_norm)

        # Add noise to gradients
        add_noise_to_gradients(net.parameters(), noise_multiplier, max_grad_norm)

        optimizer.step()
        running_loss += loss.item()
        if i % 50 == 49:  # Print every 50 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 50:.3f}')
            running_loss = 0.0

    # Validate the network on the validation set
    net.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    val_accuracy = 100 * val_correct / val_total
    val_loss /= len(testloader)
    print(f'Validation Accuracy after epoch {epoch + 1}: {val_accuracy:.2f} %')
    scheduler.step(val_loss)

torch.save(net.state_dict(), 'net_private.pth')


save_model(net, 'cifar10_classifier_Q5.pth')
zip_model('cifar10_classifier_Q5.pth', 'cifar10_classifier_Q5.zip')
upload_to_drive('cifar10_classifier_Q5.zip', '/content/drive/MyDrive/cifar10_classifier_Q5.zip')

download_from_drive('/content/drive/MyDrive/cifar10_classifier_Q5.zip', 'cifar10_classifier_Q5.zip')
unzip_model('cifar10_classifier_Q5.zip', './')
net = CIFAR10Classifier().to(device)
net = load_model(net, 'cifar10_classifier_Q5.pth')
net.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f} %')

Using device: cuda 
Files already downloaded and verified 
Files already downloaded and verified 
[Epoch 1, Batch 50] loss: 2.306 
[Epoch 1, Batch 100] loss: 2.297 
[Epoch 1, Batch 150] loss: 2.292 
[Epoch 1, Batch 200] loss: 2.286 
[Epoch 1, Batch 250] loss: 2.278 
[Epoch 1, Batch 300] loss: 2.274 
[Epoch 1, Batch 350] loss: 2.266 
Validation Accuracy after epoch 1: 22.36 % 
[Epoch 2, Batch 50] loss: 2.259 
[Epoch 2, Batch 100] loss: 2.255 
[Epoch 2, Batch 150] loss: 2.245 
[Epoch 2, Batch 200] loss: 2.235 
[Epoch 2, Batch 250] loss: 2.238 
[Epoch 2, Batch 300] loss: 2.232 
[Epoch 2, Batch 350] loss: 2.230 
Validation Accuracy after epoch 2: 25.32 % 
[Epoch 3, Batch 50] loss: 2.228 
[Epoch 3, Batch 100] loss: 2.221 
[Epoch 3, Batch 150] loss: 2.221 
[Epoch 3, Batch 200] loss: 2.213 
[Epoch 3, Batch 250] loss: 2.207 
[Epoch 3, Batch 300] loss: 2.212 
[Epoch 3, Batch 350] loss: 2.213 
Validation Accuracy after epoch 3: 26.47 % 
[Epoch 4, Batch 50] loss: 2.205 
[Epoch 4, Batch 100] loss:

# **Simulation Question 6.**
Train two Attacker Models based on MIA techniques learned
in Phase 0, one for the baseline model and one for the modified model. Compare the MIA
accuracy of these two attacker models. Use 80 percent of the training data as your seen data,
and the remaining training data along with the test data as your unseen data

In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import random
from google.colab import drive
import shutil
import zipfile
import torch.nn as nn
import torch.nn.functional as F
from model import CIFAR10Classifier

def load_model(model, path='cifar10_classifier.pth'):
    state_dict = torch.load(path)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    print(f'Model loaded from {path}')
    return model

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Data Augmentation and Normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = random_split(trainset, [train_size, val_size])

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Download the baseline model from Google Drive and unzip it
download_from_drive('/content/drive/MyDrive/cifar10_classifier.zip', 'cifar10_classifier.zip')
unzip_model('cifar10_classifier.zip', './')

# Load the baseline model
baseline_model = CIFAR10Classifier().to(device)
baseline_model = load_model(baseline_model, 'cifar10_classifier.pth')

# Download the modified model from Google Drive and unzip it
download_from_drive('/content/drive/MyDrive/cifar10_classifier_Q5.zip', 'cifar10_classifier_Q5.zip')
unzip_model('cifar10_classifier_Q5.zip', './')

# Load the modified model
modified_model = CIFAR10Classifier().to(device)
modified_model = load_model(modified_model, 'cifar10_classifier_Q5.pth')

Using device: cuda 
Files already downloaded and verified 
Files already downloaded and verified 
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True). 
Resizing fc1.weight 
Model loaded from cifar10_classifier.pth 
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True). 
Resizing fc1.weight 
Model loaded from cifar10_classifier_Q5.pth


In [ ]:
def train_shadow_models(trainloader, num_shadow_models, model):
    shadow_models = []
    for _ in range(num_shadow_models):
        shadow_model = CIFAR10Classifier().to(device)
        shadow_model.load_state_dict(model.state_dict())
        shadow_model.train()
        optimizer = optim.SGD(shadow_model.parameters(), lr=0.01, momentum=0.9)
        for epoch in range(5):
            for data in trainloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = shadow_model(inputs)
                loss = F.cross_entropy(outputs, labels)
                loss.backward()
                optimizer.step()
        shadow_models.append(shadow_model)
    return shadow_models

In [ ]:
def extract_features(shadow_models, data_loader):
    features = []
    for shadow_model in shadow_models:
        shadow_model.eval()
        for data in data_loader:
            inputs, _ = data
            inputs = inputs.to(device)
            with torch.no_grad():
                outputs = shadow_model(inputs)
            features.extend(outputs.cpu().numpy())
    return np.array(features)

In [ ]:
def train_attacker_model(features, labels):
    features = torch.tensor(features, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    input_dim = features.shape[1]
    attacker_model = nn.Sequential(
        nn.Linear(input_dim, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    ).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(attacker_model.parameters(), lr=0.001)
    for epoch in range(20):
        attacker_model.train()
        optimizer.zero_grad()
        outputs = attacker_model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    return attacker_model


In [ ]:
# Train shadow models for both baseline and private models
shadow_models_baseline = train_shadow_models(trainloader, 20, baseline_model)
shadow_models_private = train_shadow_models(trainloader, 20, modified_model)

In [ ]:

features_baseline = extract_features(shadow_models_baseline, trainloader)
features_private = extract_features(shadow_models_private, trainloader)

In [ ]:
labels_baseline = [1 if i < len(features_baseline) / 2 else 0 for i in range(len(features_baseline))]
labels_private = [1 if i < len(features_private) / 2 else 0 for i in range(len(features_private))]


In [ ]:
attacker_model_baseline = train_attacker_model(features_baseline, labels_baseline)
attacker_model_private = train_attacker_model(features_private, labels_private)


In [ ]:

# Evaluate the attacker models
def evaluate_attacker_model(attacker_model, features, labels):
    features = torch.tensor(features, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        outputs = attacker_model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return correct / total

In [11]:
# Evaluate attacker models
mia_accuracy_baseline = evaluate_attacker_model(attacker_model_baseline, features_baseline, labels_baseline)
mia_accuracy_private = evaluate_attacker_model(attacker_model_private, features_private, labels_private)

print(f'Baseline Model MIA Accuracy: {mia_accuracy_baseline * 100:.2f}%')
print(f'Private Model MIA Accuracy: {mia_accuracy_private * 100:.2f}%')

Baseline Model MIA Accuracy: 57.27% 
Private Model MIA Accuracy: 56.63%


# **Simulation Question 7.**
Improve your attacker models to achieve better MIA accuracy for
both the baseline and modified models (e.g., by increasing the number of shadow models).
Then compare the new accuracies with the previous results.

In [12]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split, Dataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import random
from google.colab import drive
import shutil
import zipfile
import torch.nn as nn
import torch.nn.functional as F
from model import CIFAR10Classifier

def load_model(model, path='cifar10_classifier.pth'):
    state_dict = torch.load(path)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    print(f'Model loaded from {path}')
    return model


# Function to save the model
def zip_model(local_path, zip_name):
    import zipfile
    with zipfile.ZipFile(zip_name, 'w') as zipf:
        zipf.write(local_path)

# Define functions for Google Drive operations
def upload_to_drive(local_path, drive_path):
    from google.colab import drive
    drive.mount('/content/drive')
    shutil.move(local_path, drive_path)

def download_from_drive(drive_path, local_path):
    from google.colab import drive
    drive.mount('/content/drive')
    shutil.copy(drive_path, local_path)

def unzip_model(zip_name, extract_path):
    import zipfile
    with zipfile.ZipFile(zip_name, 'r') as zipf:
        zipf.extractall(extract_path)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Data Augmentation and Normalization


# Function to train shadow models
def train_shadow_models(trainloader, num_shadow_models, model, epochs=5):
    shadow_models = []
    for _ in range(num_shadow_models):
        shadow_model = CIFAR10Classifier().to(device)
        shadow_model.load_state_dict(model.state_dict())
        shadow_model.train()
        optimizer = optim.SGD(shadow_model.parameters(), lr=0.01, momentum=0.9)
        for epoch in range(epochs):
            for data in trainloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = shadow_model(inputs)
                loss = F.cross_entropy(outputs, labels)
                loss.backward()
                optimizer.step()
        shadow_models.append(shadow_model)
    return shadow_models

# Function to extract features using the shadow models
def extract_features(shadow_models, data_loader):
    features = []
    for shadow_model in shadow_models:
        shadow_model.eval()
        for data in data_loader:
            inputs, _ = data
            inputs = inputs.to(device)
            with torch.no_grad():
                outputs = shadow_model(inputs)
            features.extend(outputs.cpu().numpy())
    return np.array(features)

# Function to train the attacker model
def train_attacker_model(features, labels):
    features = torch.tensor(features, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    input_dim = features.shape[1]
    attacker_model = nn.Sequential(
        nn.Linear(input_dim, 256),
        nn.ReLU(),
        nn.Linear(256, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    ).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(attacker_model.parameters(), lr=0.001)
    for epoch in range(20):
        attacker_model.train()
        optimizer.zero_grad()
        outputs = attacker_model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    return attacker_model

Using device: cuda


In [13]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = random_split(trainset, [train_size, val_size])

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
valloader = DataLoader(valset, batch_size=128, shuffle=False, num_workers=2)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Download the baseline model from Google Drive and unzip it
download_from_drive('/content/drive/MyDrive/cifar10_classifier.zip', 'cifar10_classifier.zip')
unzip_model('cifar10_classifier.zip', './')

# Load the baseline model
baseline_model = CIFAR10Classifier().to(device)
baseline_model = load_model(baseline_model, 'cifar10_classifier.pth')

# Download the modified model from Google Drive and unzip it
download_from_drive('/content/drive/MyDrive/cifar10_classifier_Q5.zip', 'cifar10_classifier_Q5.zip')
unzip_model('cifar10_classifier_Q5.zip', './')

# Load the modified model
modified_model = CIFAR10Classifier().to(device)
modified_model = load_model(modified_model, 'cifar10_classifier_Q5.pth')

Files already downloaded and verified 
Files already downloaded and verified 
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True). 
Resizing fc1.weight 
Model loaded from cifar10_classifier.pth 
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True). 
Resizing fc1.weight 
Model loaded from cifar10_classifier_Q5.pth


In [ ]:
# Train shadow models for both baseline and private models with increased number of shadow models and epochs
shadow_models_baseline = train_shadow_models(trainloader, 30, baseline_model, epochs=7)
shadow_models_private = train_shadow_models(trainloader, 30, modified_model, epochs=7)

In [ ]:
# Extract features from shadow models
features_baseline = extract_features(shadow_models_baseline, trainloader)
features_private = extract_features(shadow_models_private, trainloader)

In [ ]:
# Generate labels for attacker model training
labels_baseline = [1 if i < len(features_baseline) / 2 else 0 for i in range(len(features_baseline))]
labels_private = [1 if i < len(features_private) / 2 else 0 for i in range(len(features_private))]

In [ ]:
# Train attacker models
attacker_model_baseline = train_attacker_model(features_baseline, labels_baseline)
attacker_model_private = train_attacker_model(features_private, labels_private)

In [14]:
# Evaluate the attacker models
def evaluate_attacker_model(attacker_model, features, labels):
    features = torch.tensor(features, dtype=torch.float32).to(device)
    labels = torch.tensor(labels, dtype=torch.long).to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        outputs = attacker_model(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return correct / total

# Evaluate attacker models
mia_accuracy_baseline = evaluate_attacker_model(attacker_model_baseline, features_baseline, labels_baseline)
mia_accuracy_private = evaluate_attacker_model(attacker_model_private, features_private, labels_private)

print(f'Baseline Model MIA Accuracy: {mia_accuracy_baseline * 100:.2f}%')
print(f'Private Model MIA Accuracy: {mia_accuracy_private * 100:.2f}%')

Baseline Model MIA Accuracy: 60.26% 
Private Model MIA Accuracy: 57.98%


# **Simulation Question 8.**

Attempt to train an attacker model for the given private model
(private_model.pth). We will test it on our dataset during the online presentation session. A
competitive bonus point is available for the best performance.
